In [ ]:
import os, sys
# enable absolute paths transversal (from notebooks folder to src folder)
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from datetime import datetime, timedelta

from pypfopt import expected_returns, EfficientFrontier
import pandas as pd
import numpy as np

import src.utils as utils
import src.macro.calculate as calculate
import src.macro.plot as plot

tickers = ['AAPL','AMZN','NVDA','MMC','GOOG','MSFT','BTC-USD','ETH-USD','XOM','BAC','V','GOLD','^GSPC']
start_date = '2010-01-01'
end_date = datetime.now() - timedelta(1)

# just grabbing the 'Adj Close' column
stock_data = utils.get_stock_data(tickers, start_date, end_date)

sp500_data = stock_data['^GSPC']
stock_data = stock_data.drop(['^GSPC'], axis=1)

stock_data.tail()

"""similar to portfolio/display.py, set the portfolio weights and calculate the performance
"""
risk_free_rate = 0.04
mu = expected_returns.mean_historical_return(stock_data)
S = utils.calculate_covariance_matrix(stock_data)

# Calculating the cumulative monthly returns of a portfolio of stocks with given weights:
min_risk, max_risk = utils.calculate_risk_extents(mu, S, risk_free_rate)
risk = (max_risk + min_risk) / 2

ef = EfficientFrontier(mu, S)
ef.efficient_risk(risk)
weights = ef.clean_weights()
weights = pd.Series(weights).reindex(stock_data.columns)
ef_returns, ef_volatility, ef_sharpe = ef.portfolio_performance(risk_free_rate)
print(f'ef weights:\n{weights}')
print(f'ef performance: {ef_returns, ef_volatility, ef_sharpe}')


daily_returns = stock_data.pct_change()
weighted_daily_returns = daily_returns.mul(weights, axis=1)
portfolio_daily_returns = weighted_daily_returns.sum(axis=1)
portfolio_monthly_returns = portfolio_daily_returns.resample('M').apply(lambda x: (1 + x).prod() - 1).dropna()
portfolio_quarterly_returns = portfolio_daily_returns.resample('Q').apply(lambda x: (1 + x).prod() - 1).dropna()
portfolio_annual_returns = portfolio_daily_returns.resample('Y').apply(lambda x: (1 + x).prod() - 1).dropna()

portfolio_cumulative_daily_returns = ((1 + portfolio_daily_returns).cumprod() - 1).dropna()
portfolio_cumulative_monthly_returns = ((1 + portfolio_monthly_returns).cumprod() - 1).dropna()
portfolio_cumulative_quarterly_returns = ((1 + portfolio_quarterly_returns).cumprod() - 1).dropna()
portfolio_cumulative_annual_returns = ((1 + portfolio_annual_returns).cumprod() - 1).dropna()

sp500_daily_returns = sp500_data.pct_change()
sp500_monthly_returns = sp500_daily_returns.resample('M').apply(lambda x: (1 + x).prod() - 1).dropna()
sp500_quarterly_returns = sp500_daily_returns.resample('Q').apply(lambda x: (1 + x).prod() - 1).dropna()
sp500_annual_returns = sp500_daily_returns.resample('Y').apply(lambda x: (1 + x).prod() - 1).dropna()

sp500_cumulative_daily_returns = ((1 + sp500_daily_returns).cumprod() - 1).dropna()
sp500_cumulative_monthly_returns = ((1 + sp500_monthly_returns).cumprod() - 1).dropna()
sp500_cumulative_quarterly_returns = ((1 + sp500_quarterly_returns).cumprod() - 1).dropna()
sp500_cumulative_annual_returns = ((1 + sp500_annual_returns).cumprod() - 1).dropna()

In [ ]:
# repeat this for the S&P500
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from datetime import datetime, timedelta

from pypfopt import expected_returns, EfficientFrontier
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import altair as alt
import pandas as pd


import src.utils as utils
import src.macro.calculate as calculate
import src.macro.plot as plot


# need a plotting function that takes in the portfolio, benchmark, and macroeconomic data and plots them all together
# need to be able to specify the time period (monthly, quarterly, annual) and loop over the factors
def plot_portfolio_vs_benchmark_vs_macro(portfolio_returns, benchmark_returns, macro_time_series, time_period, title_prefix=''):
    """Plot the portfolio returns vs the benchmark returns vs the macroeconomic data
    
    Parameters
    ----------
    portfolio_returns : pandas.Series
        The portfolio returns
    benchmark_returns : pandas.Series
        The benchmark returns
    macro_data_dict : dict
        A dictionary containing the macroeconomic data
    macro_time_series_name : str
        The name of the macroeconomic time series
    time_period : str
        The time period to plot the data for (monthly, quarterly, or annual)
    """
    
    
    # Create subplot for each macro factor
    for factor, macro_factor_series in macro_time_series.items():

        # Create a subplot with two y-axes
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # Add S&P500 return trace
        fig.add_trace(
            go.Scatter(x=benchmark_returns.index, y=benchmark_returns, mode='lines', name='S&P500 Returns'),
            secondary_y=False,
        )
        
        # Add portfolio return trace
        fig.add_trace(
            go.Scatter(x=portfolio_returns.index, y=portfolio_returns, mode='lines', name='Weighted Portfolio Returns', line=dict(color='royalblue')),
            secondary_y=False,
        )

        # Add macroeconomic factor trace
        fig.add_trace(
            go.Scatter(x=macro_factor_series.index, y=macro_factor_series, mode='lines', name=factor, line=dict(dash='dot')),
            secondary_y=True
        )

        # Set axis titles
        #fig.update_xaxes(title_text="Date")
        fig.update_yaxes(title_text="Returns %", secondary_y=False, tickformat=".1%")
        
        if 'Change' in factor or 'Rate' in factor:
            fig.update_yaxes(title_text=f'{factor} %', secondary_y=True, tickformat=".1%")
        else:
            fig.update_yaxes(title_text=factor, secondary_y=True)

        # Set title
        fig.update_layout(title_text=f'{title_prefix}Weighted Portfolio vs {title_prefix}S&P500 Returns vs {factor}<br><sup>Time Period: {time_period}</sup>')
        
        fig.show()
               
def plot_portfolio_vs_benchmark_vs_macro_altair(dict_of_dict_returns, dict_of_dict_cumulative_returns):
    # Define the list of time periods
    time_periods = ['Monthly', 'Quarterly', 'Yearly']

    # Iterate over the time periods
    for period in time_periods:
        returns = dict_of_dict_returns[period]
        cumulative_returns = dict_of_dict_cumulative_returns[period]
        
        # Iterate over the macro factors
        for factor in returns['Macro']:
            # Create a DataFrame for Altair
            df = pd.DataFrame({
                'Date': returns['Portfolio'].index,
                f'{period} Portfolio Returns': returns['Portfolio'].values,
                f'{period} S&P500 Returns': returns['S&P500'].values,
                f'{period} {factor}': returns['Macro'][factor].values,
                f'Cumulative {period} Portfolio Returns': cumulative_returns['Portfolio'].values,
                f'Cumulative {period} S&P500 Returns': cumulative_returns['S&P500'].values,
                f'Cumulative {period} {factor}': cumulative_returns['Macro'][factor].values,
            })

            # Make the Altair chart for returns
            chart_returns = alt.Chart(df).transform_fold(
                [f'{period} Portfolio Returns', f'{period} S&P500 Returns', f'{period} {factor}']
            ).mark_line().encode(
                x='Date:T',
                y=alt.Y('value:Q', title='Returns'),
                color='key:N'
            ).properties(
                title=f'{factor} - {period} Returns'
            )

            # Make the Altair chart for cumulative returns
            chart_cumulative_returns = alt.Chart(df).transform_fold(
                [f'Cumulative {period} Portfolio Returns', f'Cumulative {period} S&P500 Returns', f'Cumulative {period} {factor}']
            ).mark_line().encode(
                x='Date:T',
                y=alt.Y('value:Q', title='Cumulative Returns'),
                color='key:N'
            ).properties(
                title=f'{factor} - Cumulative {period} Returns'
            )

            # Display the charts
            chart_returns.display()
            chart_cumulative_returns.display()
                
# macro_data_dict = {'Macro Factor Name': has a dictionary with time series of the factor values
#        'Monthly': monthly_mean, -> compare this with *cumulative* monthly returns
#        'Monthly Change': -> monthly_change, compare this with monthly returns
#        'Quarterly': -> quarterly_mean, compare this with *cumulative* quarterly returns
#        'Quarterly -> Change': quarterly_change, compare this with quarterly returns 
#        'Yearly': -> yearly_mean, compare this with *cumulative* annual returns
#        'Yearly Change': -> yearly_change compare this with annual returns

start_date = '2010-01-01'
end_date = datetime.now() - timedelta(1)

macro_data_dict = calculate.get_historical_macro_data(start_date, end_date)

monthly_macro_data = {}
mom_change_in_macro_data = {}

quarterly_macro_data = {}
qoq_change_in_macro_data = {}

yearly_macro_data = {}
yoy_change_in_macro_data = {}

# parse the macro data dictionary into monthly, quarterly, and yearly data for comparison with the portfolio returns
for factor, time_bases in macro_data_dict.items():
    for time_basis, time_series in time_bases.items():
        if time_basis == 'Monthly':
            monthly_macro_data[factor] = time_series.dropna()
            
        if time_basis == 'Monthly Change': 
            mom_change_in_macro_data[factor] = time_series.dropna()
            
        if time_basis == 'Quarterly':
            quarterly_macro_data[factor] = time_series.dropna()
            
        if time_basis == 'Quarterly Change':
            qoq_change_in_macro_data[factor] = time_series.dropna()
            
        if time_basis == 'Yearly':
            yearly_macro_data[factor] = time_series.dropna()
        
        if time_basis == 'Yearly Change':
            yoy_change_in_macro_data[factor] = time_series.dropna()
            
def merge_dataframes(df_dict):
    # Start by merging the 'Portfolio' and 'S&P500' DataFrames
    portfolio_df = pd.DataFrame(df_dict['Portfolio'], columns=['Portfolio'])
    sp500_df = pd.DataFrame(df_dict['S&P500'], columns=['S&P500'])
    merged_df = portfolio_df.merge(sp500_df, left_index=True, right_index=True, how='inner')

    # Then, merge each macro factor DataFrame into the merged DataFrame
    macro_df_dict = df_dict['Macro']
    for factor in macro_df_dict:
        factor_df = pd.DataFrame(macro_df_dict[factor], columns=[factor])
        merged_df = merged_df.merge(factor_df, left_index=True, right_index=True, how='inner')

    return merged_df

def dict_to_combined_dict_of_df(monthly_dict, quarterly_dict, yearly_dict):
    # Merge data for each time period and return type
    merged_monthly_returns = merge_dataframes(monthly_dict)
    merged_quarterly_returns = merge_dataframes(quarterly_dict)
    merged_annual_returns = merge_dataframes(yearly_dict)

    # Store merged data in the original data structures
    returns_data = {
        'Monthly': merged_monthly_returns,
        'Quarterly': merged_quarterly_returns,
        'Yearly': merged_annual_returns,
    }
    
    return returns_data

monthly_returns = {'Portfolio': portfolio_monthly_returns, 'S&P500': sp500_monthly_returns, 'Macro': mom_change_in_macro_data} # compare macro with *cumulative* monthly returns
quarterly_returns = {'Portfolio': portfolio_quarterly_returns, 'S&P500': sp500_quarterly_returns, 'Macro': qoq_change_in_macro_data} # compare macro with *cumulative* quarterly returns
annual_returns = {'Portfolio': portfolio_annual_returns, 'S&P500': sp500_annual_returns, 'Macro': yoy_change_in_macro_data} # compare macro with *cumulative* annual returns

cumulative_monthly_returns = {'Portfolio': portfolio_cumulative_monthly_returns, 'S&P500': sp500_cumulative_monthly_returns, 'Macro': monthly_macro_data} # compare macro with change in monthly returns
cumulative_quarterly_returns = {'Portfolio': portfolio_cumulative_quarterly_returns, 'S&P500': sp500_cumulative_quarterly_returns, 'Macro': quarterly_macro_data} # compare macro with change in quarterly returns
cumulative_annual_returns = {'Portfolio': portfolio_cumulative_annual_returns, 'S&P500': sp500_cumulative_annual_returns, 'Macro': yoy_change_in_macro_data} # compare macro with change in annual returns

plot_portfolio_vs_benchmark_vs_macro(monthly_returns['Portfolio'], monthly_returns['S&P500'], monthly_returns['Macro'], 'Monthly', title_prefix='')
plot_portfolio_vs_benchmark_vs_macro(quarterly_returns['Portfolio'], quarterly_returns['S&P500'], quarterly_returns['Macro'], 'Quarterly', title_prefix='')
plot_portfolio_vs_benchmark_vs_macro(annual_returns['Portfolio'], annual_returns['S&P500'], annual_returns['Macro'], 'Annual', title_prefix='')

plot_portfolio_vs_benchmark_vs_macro(cumulative_monthly_returns['Portfolio'], cumulative_monthly_returns['S&P500'], cumulative_monthly_returns['Macro'], 'Monthly', title_prefix='Cumulative ')
plot_portfolio_vs_benchmark_vs_macro(cumulative_quarterly_returns['Portfolio'], cumulative_quarterly_returns['S&P500'], cumulative_quarterly_returns['Macro'], 'Quarterly', title_prefix='Cumulative ')
plot_portfolio_vs_benchmark_vs_macro(cumulative_annual_returns['Portfolio'], cumulative_annual_returns['S&P500'], cumulative_annual_returns['Macro'], 'Annual', title_prefix='Cumulative ')

# was trying to leverage df's, just became too confusing
#combined_returns_data = dict_to_combined_dict_of_df(monthly_returns, quarterly_returns, annual_returns)
#combined_cumulative_returns_data = dict_to_combined_dict_of_df(cumulative_monthly_returns, cumulative_quarterly_returns, cumulative_annual_returns)
#plot_portfolio_vs_benchmark_vs_macro_altair(combined_returns_data, combined_cumulative_returns_data)




# was trying to leverage altair to minimize mem impact on the browser, will have to back to this
# #plot_portfolio_vs_benchmark_vs_macro_altair(returns_data, cumulative_returns_data)


In [ ]:
import glob
from datetime import datetime, timedelta
import pandas as pd
import os, sys
import json
from pypfopt import expected_returns, EfficientFrontier

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
import src.utils as utils
import src.macro.calculate as calculate
import src.macro.plot as plot

def load_latest_tuned_hyperparameters():
    # List all files that begin with "tuned_macro_hyperparameters"
    files = glob.glob('tuned_macro_hyperparameters_*.json')

    # If no files found, return None
    if not files:
        return None

    # Sort files by date
    files.sort(key=os.path.getmtime, reverse=True)

    # Load the most recent file
    latest_file = files[0]
    with open(latest_file, 'r') as f:
        hyper_parameter_dict = json.load(f)

    return hyper_parameter_dict

date = datetime.now().strftime("%Y%m%d")
print(f'date: {date}')

cur_tuned_params_dict = load_latest_tuned_hyperparameters()
print(f'cur_tuned_params_df:\n{cur_tuned_params_dict}')

def get_combined_returns_data_v0(tickers, start_date, end_date):


    # just grabbing the 'Adj Close' column
    stock_data = utils.get_stock_data(tickers, start_date, end_date)

    sp500_data = stock_data['^GSPC']
    stock_data = stock_data.drop(['^GSPC'], axis=1)

    stock_data.tail()

    """similar to portfolio/display.py, set the portfolio weights and calculate the performance
    """
    risk_free_rate = 0.04
    mu = expected_returns.mean_historical_return(stock_data)
    S = utils.calculate_covariance_matrix(stock_data)

    # Calculating the cumulative monthly returns of a portfolio of stocks with given weights:
    min_risk, max_risk = utils.calculate_risk_extents(mu, S, risk_free_rate)
    risk = (max_risk + min_risk) / 2

    ef = EfficientFrontier(mu, S)
    ef.efficient_risk(risk)
    weights = ef.clean_weights()
    weights = pd.Series(weights).reindex(stock_data.columns)
    ef_returns, ef_volatility, ef_sharpe = ef.portfolio_performance(risk_free_rate)
    print(f'ef weights:\n{weights}')
    print(f'ef performance: {ef_returns, ef_volatility, ef_sharpe}')


    daily_returns = stock_data.pct_change()
    weighted_daily_returns = daily_returns.mul(weights, axis=1)
    portfolio_daily_returns = weighted_daily_returns.sum(axis=1)
    portfolio_monthly_returns = portfolio_daily_returns.resample('MS').apply(lambda x: (1 + x).prod() - 1).dropna()
    portfolio_quarterly_returns = portfolio_daily_returns.resample('QS').apply(lambda x: (1 + x).prod() - 1).dropna()
    portfolio_annual_returns = portfolio_daily_returns.resample('YS').apply(lambda x: (1 + x).prod() - 1).dropna()

    portfolio_cumulative_daily_returns = ((1 + portfolio_daily_returns).cumprod() - 1).dropna()
    portfolio_cumulative_monthly_returns = ((1 + portfolio_monthly_returns).cumprod() - 1).dropna()
    portfolio_cumulative_quarterly_returns = ((1 + portfolio_quarterly_returns).cumprod() - 1).dropna()
    portfolio_cumulative_annual_returns = ((1 + portfolio_annual_returns).cumprod() - 1).dropna()

    sp500_daily_returns = sp500_data.pct_change()
    sp500_monthly_returns = sp500_daily_returns.resample('MS').apply(lambda x: (1 + x).prod() - 1).dropna()
    sp500_quarterly_returns = sp500_daily_returns.resample('QS').apply(lambda x: (1 + x).prod() - 1).dropna()
    sp500_annual_returns = sp500_daily_returns.resample('YS').apply(lambda x: (1 + x).prod() - 1).dropna()

    sp500_cumulative_daily_returns = ((1 + sp500_daily_returns).cumprod() - 1).dropna()
    sp500_cumulative_monthly_returns = ((1 + sp500_monthly_returns).cumprod() - 1).dropna()
    sp500_cumulative_quarterly_returns = ((1 + sp500_quarterly_returns).cumprod() - 1).dropna()
    sp500_cumulative_annual_returns = ((1 + sp500_annual_returns).cumprod() - 1).dropna()    
    
    macro_data_dict = calculate.get_historical_macro_data(start_date, end_date)

    monthly_macro_data = {}
    mom_change_in_macro_data = {}

    quarterly_macro_data = {}
    qoq_change_in_macro_data = {}

    yearly_macro_data = {}
    yoy_change_in_macro_data = {}

    # parse the macro data dictionary into monthly, quarterly, and yearly data for comparison with the portfolio returns
    for factor, time_bases in macro_data_dict.items():
        for time_basis, time_series in time_bases.items():
            if time_basis == 'Monthly':
                monthly_macro_data[factor] = time_series.dropna()
                
            if time_basis == 'Monthly Change': 
                mom_change_in_macro_data[factor] = time_series.dropna()
                
            if time_basis == 'Quarterly':
                quarterly_macro_data[factor] = time_series.dropna()
                
            if time_basis == 'Quarterly Change':
                qoq_change_in_macro_data[factor] = time_series.dropna()
                
            if time_basis == 'Yearly':
                yearly_macro_data[factor] = time_series.dropna()
            
            if time_basis == 'Yearly Change':
                yoy_change_in_macro_data[factor] = time_series.dropna()
                
            print(f'{factor} {time_basis}:\n{time_series.head()}\n')
            
    monthly_returns = {'Portfolio': portfolio_monthly_returns, 'S&P500': sp500_monthly_returns, 'Macro': mom_change_in_macro_data} # compare macro with *cumulative* monthly returns
    quarterly_returns = {'Portfolio': portfolio_quarterly_returns, 'S&P500': sp500_quarterly_returns, 'Macro': qoq_change_in_macro_data} # compare macro with *cumulative* quarterly returns
    annual_returns = {'Portfolio': portfolio_annual_returns, 'S&P500': sp500_annual_returns, 'Macro': yoy_change_in_macro_data} # compare macro with *cumulative* annual returns

    cumulative_monthly_returns = {'Portfolio': portfolio_cumulative_monthly_returns, 'S&P500': sp500_cumulative_monthly_returns, 'Macro': monthly_macro_data} # compare macro with change in monthly returns
    cumulative_quarterly_returns = {'Portfolio': portfolio_cumulative_quarterly_returns, 'S&P500': sp500_cumulative_quarterly_returns, 'Macro': quarterly_macro_data} # compare macro with change in quarterly returns
    cumulative_annual_returns = {'Portfolio': portfolio_cumulative_annual_returns, 'S&P500': sp500_cumulative_annual_returns, 'Macro': yoy_change_in_macro_data} # compare macro with change in annual returns

    returns_data = {
        'Monthly': monthly_returns,
        'Quarterly': quarterly_returns,
        'Yearly': annual_returns,
    }

    cumulative_returns_data = {
        'Monthly': cumulative_monthly_returns,
        'Quarterly': cumulative_quarterly_returns,
        'Yearly': cumulative_annual_returns,
    }

    return returns_data, cumulative_returns_data

def resample_from_daily_stock_returns_series(daily_returns_series):    
    portfolio_monthly_returns = portfolio_daily_returns.resample('MS').apply(lambda x: (1 + x).prod() - 1).dropna()
    portfolio_quarterly_returns = portfolio_daily_returns.resample('QS').apply(lambda x: (1 + x).prod() - 1).dropna()
    portfolio_annual_returns = portfolio_daily_returns.resample('YS').apply(lambda x: (1 + x).prod() - 1).dropna()

    portfolio_cumulative_daily_returns = ((1 + portfolio_daily_returns).cumprod() - 1).dropna()
    portfolio_cumulative_monthly_returns = ((1 + portfolio_monthly_returns).cumprod() - 1).dropna()
    portfolio_cumulative_quarterly_returns = ((1 + portfolio_quarterly_returns).cumprod() - 1).dropna()
    portfolio_cumulative_annual_returns = ((1 + portfolio_annual_returns).cumprod() - 1).dropna()
    
    # establish "Change" to imply daily, monthly, etc. returns
    # establish w/o "Change" to imply cumulative returns
    return {
        'Daily': portfolio_cumulative_daily_returns, 
        'Daily Change': portfolio_daily_returns,
        'Monthly': portfolio_cumulative_monthly_returns, 
        'Monthly Change': portfolio_monthly_returns, 
        'Quarterly': portfolio_cumulative_quarterly_returns, 
        'Quarterly Change': portfolio_quarterly_returns, 
        'Yearly': portfolio_cumulative_annual_returns, 
        'Yearly Change': portfolio_annual_returns
    }
    
def get_historical_portfolio_weighted_returns_data(tickers, start_date, end_date, risk_free_rate=0.04):
    # just grabbing the 'Adj Close' column
    stock_data = utils.get_stock_data(tickers, start_date, end_date)

    """similar to portfolio/display.py, set the portfolio weights and calculate the performance
    """
    mu = expected_returns.mean_historical_return(stock_data)
    S = utils.calculate_covariance_matrix(stock_data)

    # Calculating the cumulative monthly returns of a portfolio of stocks with given weights:
    min_risk, max_risk = utils.calculate_risk_extents(mu, S, risk_free_rate)
    risk = (max_risk + min_risk) / 2 # set to mid point risk? or max risk?

    ef = EfficientFrontier(mu, S)
    ef.efficient_risk(risk)
    weights = ef.clean_weights()
    weights = pd.Series(weights).reindex(stock_data.columns)
    ef_returns, ef_volatility, ef_sharpe = ef.portfolio_performance(risk_free_rate)
    print(f'ef weights:\n{weights}')
    print(f'ef performance: {ef_returns, ef_volatility, ef_sharpe}')

    daily_returns = stock_data.pct_change()
    weighted_daily_returns = daily_returns.mul(weights, axis=1)
    portfolio_daily_returns = weighted_daily_returns.sum(axis=1)
        
    # return ticker list with non-zero weights
    portfolio_tickers = [ticker for ticker, weight in weights.items() if weight > 0]
    
    portfolio_returns_dict = resample_from_daily_stock_returns_series(portfolio_daily_returns)
        
    return portfolio_returns_dict, portfolio_tickers

        
def get_combined_returns_data(tickers, start_date, end_date):
   
    sp500_data = utils.get_stock_data(['^GSPC'], start_date, end_date)
    sp500_daily_returns = sp500_data.pct_change()
    sp500_returns_dict = resample_from_daily_stock_returns_series(sp500_daily_returns)

    portfolio_returns_dict, portfolio_tickers = get_historical_portfolio_weighted_returns_data(tickers, start_date, end_date)
    macro_data_dict = calculate.get_historical_macro_data(start_date, end_date)

    monthly_macro_data = {}
    mom_change_in_macro_data = {}

    quarterly_macro_data = {}
    qoq_change_in_macro_data = {}

    yearly_macro_data = {}
    yoy_change_in_macro_data = {}
       
    for factor, time_bases in macro_data_dict.items():
        for time_basis, time_series in time_bases.items():
            if time_basis == 'Monthly':
                monthly_macro_data[factor] = time_series.dropna()
                
            if time_basis == 'Monthly Change': 
                mom_change_in_macro_data[factor] = time_series.dropna()
                
            if time_basis == 'Quarterly':
                quarterly_macro_data[factor] = time_series.dropna()
                
            if time_basis == 'Quarterly Change':
                qoq_change_in_macro_data[factor] = time_series.dropna()
                
            if time_basis == 'Yearly':
                yearly_macro_data[factor] = time_series.dropna()
            
            if time_basis == 'Yearly Change':
                yoy_change_in_macro_data[factor] = time_series.dropna()
                
            print(f'{factor} {time_basis}:\n{time_series.head()}\n')
            
    # what to do with the daily returns? maybe return the portfolio and benchmark daily returns separately?
    monthly_returns = {'Portfolio': portfolio_returns_dict['Monthly Change'], 'S&P500': sp500_returns_dict['Monthly Change'], 'Macro': mom_change_in_macro_data} # compare macro with monthly returns
    quarterly_returns = {'Portfolio': portfolio_returns_dict['Quarterly Change'], 'S&P500': sp500_returns_dict['Quarterly Change'], 'Macro': qoq_change_in_macro_data} # compare macro with quarterly returns
    annual_returns = {'Portfolio': portfolio_returns_dict['Yearly Change'], 'S&P500': sp500_returns_dict['Yearly Change'], 'Macro': yoy_change_in_macro_data} # compare macro with annual returns

    cumulative_monthly_returns = {'Portfolio': portfolio_returns_dict['Monthly'], 'S&P500': sp500_returns_dict['Monthly'], 'Macro': monthly_macro_data} # compare macro with cumulative monthly returns
    cumulative_quarterly_returns = {'Portfolio': portfolio_returns_dict['Quarterly'], 'S&P500': sp500_returns_dict['Quarterly'], 'Macro': quarterly_macro_data} # compare macro with cumulative quarterly returns
    cumulative_annual_returns = {'Portfolio': portfolio_returns_dict['Yearly'], 'S&P500': sp500_returns_dict['Yearly'], 'Macro': yoy_change_in_macro_data} # compare macro with cumulative annual returns

    combined_returns_data = {
        'Monthly': monthly_returns,
        'Quarterly': quarterly_returns,
        'Yearly': annual_returns,
    }

    combined_cumulative_returns_data = {
        'Monthly': cumulative_monthly_returns,
        'Quarterly': cumulative_quarterly_returns,
        'Yearly': cumulative_annual_returns,
    }

    return combined_returns_data, combined_cumulative_returns_data, portfolio_returns_dict, sp500_returns_dict, macro_data_dict, portfolio_tickers

tickers = ['AAPL','AMZN','NVDA','MMC','GOOG','MSFT','BTC-USD','ETH-USD','XOM','BAC','V','GOLD','^GSPC']
start_date = '2010-01-01'
end_date = datetime.now() - timedelta(1)
    
returns_data, cumulative_returns_data, portfolio_returns_dict, sp500_returns_dict, macro_data_dict, portfolio_tickers = get_combined_returns_data(tickers, start_date, end_date)

for time_basis, returns in returns_data.items():
    print(f'Portfolio {time_basis} returns index:\n{type(returns["Portfolio"].index)}\n')
    for factor, series in returns['Macro'].items():
        print(f'Macro {factor} {time_basis} returns index:\n{type(series.index)}\n')
    
for time_basis, returns in cumulative_returns_data.items():
    print(f'Portfolio {time_basis} cumulative returns index:\n{type(returns["Portfolio"].index)}\n')
    for factor, series in returns['Macro'].items():
        print(f'Macro {factor} {time_basis} cumulative returns index:\n{type(series.index)}\n')
    




VAR processing: Prepare the data: You would first make sure that your macroeconomic data and returns data are of the same frequency (monthly in this case) and have the same time period. It's also important to ensure that there are no missing values.

Stationarity check: The VAR model requires the data to be stationary. Therefore, you would need to check if the data is stationary or not. If not, differences of the series might need to be taken.

Choose the order of the VAR model: You would need to select the lag order for the VAR model. This could be done using various statistical criteria like the Akaike Information Criterion (AIC) or the Bayesian Information Criterion (BIC).

Fit the VAR model: Once the data is ready and the order is selected, you can fit the VAR model to your data. This would give you the parameters of the model.

Check for serial correlation of residuals: After fitting the VAR model, you would want to check if there's any autocorrelation in the residuals because the residuals of a well-specified VAR are supposed to be white noise. If there's autocorrelation, the model might be mis-specified.

Forecasting: Once the VAR model is ready and validated, you can use it to make forecasts about your portfolio returns and macroeconomic factors for the next period.

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.api import OLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import ccf

import plotly.graph_objects as go
import plotly.subplots as sp
import colorsys

# Aligning the dataframes
def align_dataframes(dfs):
    # Get the common datetime index
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)
    
    # Align all dataframes to the common index
    aligned_dfs = [df.loc[common_index] for df in dfs]
    return aligned_dfs

def prepare_data(returns_data):
    portfolio_returns = returns_data['Monthly']['Portfolio'].to_frame(name='Portfolio')
    sp500_returns = returns_data['Monthly']['S&P500'].to_frame(name='S&P500')

    # concat sp500 with portfolio returns
    portfolio_returns = pd.concat([portfolio_returns, sp500_returns], axis=1)

    # Loop through all macro factors and add to DataFrame
    for factor_name, factor_df in returns_data['Monthly']['Macro'].items():
        factor_df = factor_df.rename(factor_name)  # Rename the series
        portfolio_returns = pd.concat([portfolio_returns, factor_df], axis=1)

    # Use align_dataframes to ensure the data is properly aligned
    aligned_data = align_dataframes([portfolio_returns])

    aligned_data[0].index = pd.to_datetime(aligned_data[0].index)
    aligned_data[0] = aligned_data[0].dropna()
    
    return aligned_data[0]  # align_dataframes returns a list, so we take the first element

"""
The check_multicollinearity() function calculates the Variance Inflation Factor (VIF) for each macro factor. 
VIF measures the correlation between each factor and all other factors. If a factor's VIF exceeds a certain 
threshold (in this case, 5), it indicates high multicollinearity.
"""
def check_multicollinearity(df, vif_threshold=5):
    df = df.copy()
    df.drop('Portfolio',axis=1,inplace=True)
    
    variables = df.columns
    vif_df = pd.DataFrame()
    vif_df["VIF"] = [variance_inflation_factor(df[variables].values, df.columns.get_loc(var)) for var in df.columns]
    vif_df["Features"] = variables

    columns_to_drop = []
    while vif_df['VIF'].max() > vif_threshold:
        remove = vif_df.sort_values('VIF',ascending=False)['Features'][:1]
        columns_to_drop.append(remove.values[0])
        df.drop(remove,axis=1,inplace=True)
        variables = df.columns
        vif_df = pd.DataFrame()
        vif_df["VIF"] = [variance_inflation_factor(df[variables].values, df.columns.get_loc(var)) for var in df.columns]
        vif_df["Features"] = variables
    return columns_to_drop

def get_factor_list(df):
    factor_list = list(df.columns)
    factor_list.remove('Portfolio')
    factor_list.remove('S&P500')
    return factor_list

def calculate_linear_regression_model_for_factor(input_df, factor):
    df = input_df.copy()
    df = df.dropna(how='any').reset_index(drop=True)
    X = df[[factor]]
    y = df['Portfolio']
    X2 = add_constant(X)
    model = OLS(y, X2).fit()
    coefficient = model.params[factor]
    p_value = model.pvalues[factor]
    return model, coefficient, p_value

def calculate_multivariate_analysis(input_df):
    df = input_df.copy()
    df = df.dropna(how='any').reset_index(drop=True)
    X = df[get_factor_list(df)]  
    y = df['Portfolio']
    
    model = OLS(y, add_constant(X)).fit()
    
    # Extract the significant features based on p-values
    significant_features = model.pvalues[model.pvalues < 0.05].index.tolist()
    
    if 'const' in significant_features:
        significant_features.remove('const')
        
    return model, significant_features

def plot_multivariate_results(input_df, multivariate_model, significant_features):
    fig = go.Figure(data=[go.Bar(name='Coefficient', x=multivariate_model.params.index, y=multivariate_model.params.values), 
                          go.Bar(name='p-value', x=multivariate_model.pvalues.index, y=multivariate_model.pvalues.values)])
    fig.update_layout(barmode='group', title_text='Multivariate Regression Results')
    fig.show()

    # Scatter plots for each significant feature
    for feature in significant_features:
        # Calculate the OLS trendline
        m, b = np.polyfit(input_df[feature], input_df['Portfolio'], 1)

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=input_df[feature], y=input_df['Portfolio'], mode='markers', name='observations'))
        fig2.add_trace(go.Scatter(x=input_df[feature], y=m*input_df[feature] + b, mode='lines', name='OLS trendline'))
        fig2.update_layout(title=f'Portfolio Returns vs {feature}', xaxis_title=feature, yaxis_title='Portfolio Returns')
        fig2.show()

def plot_linear_regression_results_v0(regression_model_dict):
    factors = list(regression_model_dict.keys())
    coefficients = [regression_model_dict[factor]['linear']['coefficient'] for factor in factors]
    p_values = [regression_model_dict[factor]['linear']['p_value'] for factor in factors]

    # Order by p-values
    ordered_indices = np.argsort(p_values)
    factors_ordered = np.array(factors)[ordered_indices]
    coefficients_ordered = np.array(coefficients)[ordered_indices]
    p_values_ordered = np.array(p_values)[ordered_indices]

    fig = sp.make_subplots(rows=2, cols=1, subplot_titles=("Coefficients", "p-values"), vertical_spacing=0.1)

    # Coefficients
    fig.add_trace(
        go.Bar(name='Coefficient', x=factors_ordered, y=coefficients_ordered, marker=dict(color=coefficients_ordered, colorscale='balance', showscale=True)), 
        row=1, col=1
    )
    
    # P-values
    fig.add_trace(
        go.Bar(name='p-value', x=factors_ordered, y=p_values_ordered), 
        row=2, col=1
    )

    # add line at p=0.05
    fig.add_shape(type="line", x0=-0.5, y0=0.05, x1=len(factors)-0.5, y1=0.05, yref="y2", line=dict(color="red", dash="dash"))

    fig.update_layout(title_text='Linear Regression Results', showlegend=False)

    fig.show()

def plot_linear_regression_results(regression_models_df):
    factors = regression_models_df['Factor'].tolist()
    coefficients = regression_models_df['Coefficient'].tolist()
    p_values = regression_models_df['p-value'].tolist()

    # Order by p-values
    ordered_indices = np.argsort(p_values)
    factors_ordered = np.array(factors)[ordered_indices]
    coefficients_ordered = np.array(coefficients)[ordered_indices]
    p_values_ordered = np.array(p_values)[ordered_indices]

    fig = sp.make_subplots(rows=2, cols=1, subplot_titles=("Coefficients", "p-values"), vertical_spacing=0.1)

    # Coefficients
    fig.add_trace(
        go.Bar(name='Coefficient', x=factors_ordered, y=coefficients_ordered, marker=dict(color=coefficients_ordered, colorscale='balance', showscale=True)), 
        row=1, col=1
    )
    
    # P-values
    fig.add_trace(
        go.Bar(name='p-value', x=factors_ordered, y=p_values_ordered), 
        row=2, col=1
    )

    # add line at p=0.05
    fig.add_shape(type="line", x0=-0.5, y0=0.05, x1=len(factors)-0.5, y1=0.05, yref="y2", line=dict(color="red", dash="dash"))

    fig.update_layout(title_text='Linear Regression Results', showlegend=False)

    fig.show()
    
"""
1. Portfolio Performance vs Factor over Time: This plot shows the portfolio returns and the given factor (e.g., S&P 500, unemployment rate, GDP growth, etc.) over time. 
You want to see if there are any visible patterns or correlations. Do they seem to move together (i.e., both go up or down at the same time)? Do they move in opposite 
directions? If there's a visible relationship, that's a good sign that your factor may be a useful predictor. Of course, you'd have to validate this with more rigorous 
statistical tests.

2. Scatter of Portfolio Returns vs Factor: This plot shows each observation of your portfolio returns against the factor. You're looking for any kind of relationship - 
linear, non-linear, or no relationship at all. A linear relationship where the points cluster around a straight line (either increasing or decreasing) is a good sign 
that linear regression may be a good model. If you see a different pattern, like a curve or clusters, a different model might be more appropriate.

3. Linear Regression Model Performance Prediction vs Actual Values: In this plot, you want the predicted values to be as close to the actual values as possible. In other 
words, the dots should be close to the fitted line. If the points scatter widely around the line, your model might not be a great fit.

4. Regression Model Residuals (error) vs Predicted Values: This plot shows the residuals (errors) of your model against the predicted values. In a well-performing model, 
you'd expect to see the residuals scattered randomly around the zero line. If you see patterns in the residuals, like a curve or a funnel shape, it's a sign that your 
model isn't capturing some aspect of the data.

In terms of mathematical measures:

R-squared: This is a measure of how well your model fits the data. It's a number between 0 and 1, where 1 means a perfect fit and 0 means no fit at all. Generally, a 
higher R-squared indicates a better fit, but beware of overfitting if your R-squared is very close to 1.

Correlation: This is a measure of the linear relationship between two variables. It ranges from -1 to 1. A positive correlation means that when one variable increases, 
the other tends to increase, too. A negative correlation means that when one variable increases, the other tends to decrease. A correlation close to 0 means there's no 
linear relationship.

Remember that correlation does not imply causation - just because two variables move together doesn't mean one is causing the other to move. It could be a coincidence, 
or there could be a third factor causing both to move.

Regression coefficients (slope and intercept): These numbers tell you the exact mathematical relationship between your predictor and the response. The slope tells you 
how much the response is expected to change when the predictor increases by one unit. The intercept tells you the expected response when the predictor is zero.
These are some of the key things to look for when performing linear regression analysis. Each situation is unique, and you may need to use other techniques and consider 
other factors depending on your specific dataset and research question.

"""
def create_linear_regression_plots_v0(df, factor, regression_models_dict, time_basis, cumulative_performance):
    linear_regression_model = regression_models_dict[factor]['linear']['model']
    
    X = df[[factor]]
    y = df['Portfolio']
    X2 = add_constant(X)
    predictions = linear_regression_model.predict(X2)
    residuals = y - predictions

    # Intercept and slope for the annotation
    intercept = linear_regression_model.params['const']
    slope = linear_regression_model.params[factor]

    # Correlation between the factor and the portfolio returns
    correlation = df["Portfolio"].corr(df[factor])

    if cumulative_performance:
        # Initialize subplots
        fig = sp.make_subplots(rows=1, cols=4, subplot_titles=(f"Portfolio Returns vs<br>{factor} over Time", f"Scatter of Portfolio Returns vs<br>{factor}", f"Model Prediction of Returns vs<br>{factor}", f"Regression Model Residuals (error) vs<br>{factor}"),  specs=[[{'secondary_y': True}, {'secondary_y': True}, {'secondary_y': True}, {}]])
    else:
        fig = sp.make_subplots(rows=1, cols=4, subplot_titles=(f"Portfolio Returns vs<br>{time_basis} Change of {factor}", f"Scatter of Portfolio Returns vs<br>{time_basis} Change of {factor}", f"Model Prediction of Returns vs<br>{time_basis} Change of {factor}", f"Regression Model Residuals (error) vs<br>{time_basis} Change of {factor}"),  specs=[[{'secondary_y': True}, {'secondary_y': True}, {'secondary_y': True}, {}]])

    # Subplot 1: Portfolio vs Factor over time
    fig.add_trace(go.Scatter(x=df.index, y=df["Portfolio"], mode='lines', name='Portfolio', line=dict(color='royalblue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df[factor], mode='lines', name=factor, line=dict(dash='dot')), secondary_y=True, row=1, col=1)
   
    # Set range based on respective variable - if not cumulative performance we're comparing month to month percetange change, center the y-axis at 0
    if cumulative_performance:
        # we're comparing cumulative returns which cannot be negative, so let the range be what it is
        fig.update_yaxes(title_text="Actual Returns", tickformat=".1%", row=1, col=1)
        if 'Rate' in factor:
            fig.update_yaxes(title_text=factor, tickformat=".1%", secondary_y=True, row=1, col=1)
        else:
            fig.update_yaxes(title_text=factor, tickformat="", secondary_y=True, row=1, col=1)        
    else:
        # we're comparing month to month percetange change, center the y-axis at 0
        y_range = max(abs(df["Portfolio"])) + 0.1*max(abs(df["Portfolio"])) # max absolute value of portfolio returns + 5% buffer for visuals
        secondary_y_range = max(abs(df[factor])) + 0.1*max(abs(df[factor]))
        
        fig.update_yaxes(title_text="Actual Returns", range=[-y_range, y_range], tickformat=".1%", row=1, col=1)        
        fig.update_yaxes(title_text=f'{time_basis} Change of<br>{factor}', range=[-secondary_y_range, secondary_y_range], tickformat=".1%", secondary_y=True, row=1, col=1)


    # Subplot 2: Scatter of Portfolio Returns vs Factor
    fig.add_trace(go.Scatter(x=df[factor], y=df["Portfolio"], mode='markers', name='Scatter of Factor vs Portfolio', marker=dict(size=10, color='rgba(0, 0, 152, .8)')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df[factor], y=intercept + slope * df[factor], mode='lines', name='Model Fitted line', line=dict(color='red', width=2)), row=1, col=2)
    
    # Set range based on respective variable
    if cumulative_performance:
        # cumulative returns cannot be negative, so let the range be what it is
        fig.update_yaxes(title_text="Actual Returns", tickformat=".1%", row=1, col=2)   
        if 'Rate' in factor: # add % sign to tick labels if the factor is a rate, eg, unemployment rate
            fig.update_xaxes(title_text=factor, tickformat=".1%", row=1, col=2)    
        else:
            fig.update_xaxes(title_text=factor, row=1, col=2)
    else:
        # we're comparing month to month percetange change, center the y-axis at 0
        x_range = max(abs(df[factor])) + 0.1*max(abs(df[factor])) # max absolute value of factor + 5% buffer for visuals
        y_range = max(abs(df["Portfolio"])) + 0.1*max(abs(df["Portfolio"])) # max absolute value of portfolio returns + 5% buffer for visuals
        
        fig.update_xaxes(title_text=f'{time_basis} Change of<br>{factor}', range=[-x_range, x_range], tickformat=".1%", row=1, col=2)
        fig.update_yaxes(title_text=f"Actual {time_basis} Returns", range=[-y_range, y_range], tickformat=".1%", row=1, col=2)


    # Subplot 3: Predicted Return vs Actual Factor Values 
    fig.add_trace(go.Scatter(x=df[factor], y=predictions, mode='markers', name='Model Predicted vs Actual Factor', marker=dict(size=10, color='rgba(0, 152, 0, .8)')), row=1, col=3)
    if cumulative_performance:
        # cumulative returns cannot be negative, so let the range be what it is
        fig.update_yaxes(title_text=f"Predicted Cumulative {time_basis} Returns", tickformat=".1%", row=1, col=3)
        if 'Rate' in factor:
            fig.update_xaxes(title_text=f"{factor}", tickformat=".1%", row=1, col=3)
        else:
            fig.update_xaxes(title_text=f"{factor}", row=1, col=3)
    else:
        fig.update_xaxes(title_text=f"{time_basis} Change of<br>{factor}", tickformat=".1%", row=1, col=3)
        fig.update_yaxes(title_text=f"Predicted {time_basis} Returns", tickformat=".1%", row=1, col=3)

    # Subplot 4: Residuals vs Factor Values
    fig.add_trace(go.Scatter(x=df[factor], y=residuals, mode='markers', name='Residuals vs Factor', marker=dict(size=10, color='rgba(152, 0, 0, .8)')), row=1, col=4)
    fig.update_yaxes(title_text="Residuals", tickformat=".1%", row=1, col=4)
    if cumulative_performance:
        if 'Rate' in factor:
            fig.update_xaxes(title_text=f"{factor}", tickformat=".1%", row=1, col=4)
        else:
            fig.update_xaxes(title_text=f"{factor}", row=1, col=4)
    else:
        fig.update_xaxes(title_text=f"{time_basis} Change of<br>{factor}", tickformat=".1%", row=1, col=4)

    # Update subplot titles and axis labels to be a little smaller
    for annotation in fig['layout']['annotations']: 
        annotation['font']['size']=12
    fig.update_xaxes(tickfont=dict(size=12), title_font=dict(size=12))
    fig.for_each_yaxis(lambda axis: axis.update(tickfont=dict(size=12), title_font=dict(size=12)))
    fig.add_annotation(x=0.65, y=0.95, xref="paper", yref="paper", text=f"R-squared = {linear_regression_model.rsquared:.3f}", showarrow=False, font=dict(size=8))
    fig.add_annotation(x=0.65, y=0.90, xref="paper", yref="paper", text=f"f(x) = {slope:.3f} * x + {intercept:.3f}", showarrow=False, font=dict(size=8))
    fig.add_annotation(x=0.65, y=0.85, xref="paper", yref="paper", text=f"Correlation = {correlation:.3f}", showarrow=False, font=dict(size=8))
        
    lag = regression_models_dict[factor]['linear']['optimal_lag']
    if cumulative_performance:
        fig.update_layout(title_text=f"Linear Regression Analysis and Model of {time_basis} Returns vs. {factor} (lag={lag}))")
    else:
        
        fig.update_layout(title_text=f"Linear Regression Analysis and Model of {time_basis} Returns vs. {factor} {time_basis} Change (%) (lag={lag}))")
    
    fig.show()
    
    return fig

def create_linear_regression_plots(input_df, factor, regression_models_df, time_basis, cumulative_performance):
    linear_regression_model = regression_models_df.loc[regression_models_df['Factor'] == factor, 'Model'].values[0]
    df = input_df.copy()
    df = df.dropna(how='any').reset_index(drop=True)
    
    X = df[[factor]]
    y = df['Portfolio']
    X2 = add_constant(X)
    predictions = linear_regression_model.predict(X2)
    residuals = y - predictions

    # Intercept and slope for the annotation
    intercept = linear_regression_model.params['const']
    slope = linear_regression_model.params[factor]

    # Correlation between the factor and the portfolio returns
    correlation = df["Portfolio"].corr(df[factor])

    if cumulative_performance:
        # Initialize subplots
        fig = sp.make_subplots(rows=1, cols=4, subplot_titles=(f"Portfolio Returns vs<br>{factor} over Time", f"Scatter of Portfolio Returns vs<br>{factor}", f"Model Prediction of Returns vs<br>{factor}", f"Regression Model Residuals (error) vs<br>{factor}"),  specs=[[{'secondary_y': True}, {'secondary_y': True}, {'secondary_y': True}, {}]])
    else:
        fig = sp.make_subplots(rows=1, cols=4, subplot_titles=(f"Portfolio Returns vs<br>{time_basis} Change of {factor}", f"Scatter of Portfolio Returns vs<br>{time_basis} Change of {factor}", f"Model Prediction of Returns vs<br>{time_basis} Change of {factor}", f"Regression Model Residuals (error) vs<br>{time_basis} Change of {factor}"),  specs=[[{'secondary_y': True}, {'secondary_y': True}, {'secondary_y': True}, {}]])

    # Subplot 1: Portfolio vs Factor over time
    fig.add_trace(go.Scatter(x=df.index, y=df["Portfolio"], mode='lines', name='Portfolio', line=dict(color='royalblue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df[factor], mode='lines', name=factor, line=dict(dash='dot')), secondary_y=True, row=1, col=1)
   
    # Set range based on respective variable - if not cumulative performance we're comparing month to month percetange change, center the y-axis at 0
    if cumulative_performance:
        # we're comparing cumulative returns which cannot be negative, so let the range be what it is
        fig.update_yaxes(title_text="Actual Returns", tickformat=".1%", row=1, col=1)
        if 'Rate' in factor:
            fig.update_yaxes(title_text=factor, tickformat=".1%", secondary_y=True, row=1, col=1)
        else:
            fig.update_yaxes(title_text=factor, tickformat="", secondary_y=True, row=1, col=1)        
    else:
        # we're comparing month to month percetange change, center the y-axis at 0
        y_range = max(abs(df["Portfolio"])) + 0.1*max(abs(df["Portfolio"])) # max absolute value of portfolio returns + 5% buffer for visuals
        secondary_y_range = max(abs(df[factor])) + 0.1*max(abs(df[factor]))
        
        fig.update_yaxes(title_text="Actual Returns", range=[-y_range, y_range], tickformat=".1%", row=1, col=1)        
        fig.update_yaxes(title_text=f'{time_basis} Change of<br>{factor}', range=[-secondary_y_range, secondary_y_range], tickformat=".1%", secondary_y=True, row=1, col=1)


    # Subplot 2: Scatter of Portfolio Returns vs Factor
    fig.add_trace(go.Scatter(x=df[factor], y=df["Portfolio"], mode='markers', name='Scatter of Factor vs Portfolio', marker=dict(size=10, color='rgba(0, 0, 152, .8)')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df[factor], y=intercept + slope * df[factor], mode='lines', name='Model Fitted line', line=dict(color='red', width=2)), row=1, col=2)
    
    # Set range based on respective variable
    if cumulative_performance:
        # cumulative returns cannot be negative, so let the range be what it is
        fig.update_yaxes(title_text="Actual Returns", tickformat=".1%", row=1, col=2)   
        if 'Rate' in factor: # add % sign to tick labels if the factor is a rate, eg, unemployment rate
            fig.update_xaxes(title_text=factor, tickformat=".1%", row=1, col=2)    
        else:
            fig.update_xaxes(title_text=factor, row=1, col=2)
    else:
        # we're comparing month to month percetange change, center the y-axis at 0
        x_range = max(abs(df[factor])) + 0.1*max(abs(df[factor])) # max absolute value of factor + 5% buffer for visuals
        y_range = max(abs(df["Portfolio"])) + 0.1*max(abs(df["Portfolio"])) # max absolute value of portfolio returns + 5% buffer for visuals
        
        fig.update_xaxes(title_text=f'{time_basis} Change of<br>{factor}', range=[-x_range, x_range], tickformat=".1%", row=1, col=2)
        fig.update_yaxes(title_text=f"Actual {time_basis} Returns", range=[-y_range, y_range], tickformat=".1%", row=1, col=2)


    # Subplot 3: Predicted Return vs Actual Factor Values 
    fig.add_trace(go.Scatter(x=df[factor], y=predictions, mode='markers', name='Model Predicted vs Actual Factor', marker=dict(size=10, color='rgba(0, 152, 0, .8)')), row=1, col=3)
    if cumulative_performance:
        # cumulative returns cannot be negative, so let the range be what it is
        fig.update_yaxes(title_text=f"Predicted Cumulative {time_basis} Returns", tickformat=".1%", row=1, col=3)
        if 'Rate' in factor:
            fig.update_xaxes(title_text=f"{factor}", tickformat=".1%", row=1, col=3)
        else:
            fig.update_xaxes(title_text=f"{factor}", row=1, col=3)
    else:
        fig.update_xaxes(title_text=f"{time_basis} Change of<br>{factor}", tickformat=".1%", row=1, col=3)
        fig.update_yaxes(title_text=f"Predicted {time_basis} Returns", tickformat=".1%", row=1, col=3)

    # Subplot 4: Residuals vs Factor Values
    fig.add_trace(go.Scatter(x=df[factor], y=residuals, mode='markers', name='Residuals vs Factor', marker=dict(size=10, color='rgba(152, 0, 0, .8)')), row=1, col=4)
    fig.update_yaxes(title_text="Residuals", tickformat=".1%", row=1, col=4)
    if cumulative_performance:
        if 'Rate' in factor:
            fig.update_xaxes(title_text=f"{factor}", tickformat=".1%", row=1, col=4)
        else:
            fig.update_xaxes(title_text=f"{factor}", row=1, col=4)
    else:
        fig.update_xaxes(title_text=f"{time_basis} Change of<br>{factor}", tickformat=".1%", row=1, col=4)

    # Update subplot titles and axis labels to be a little smaller
    for annotation in fig['layout']['annotations']: 
        annotation['font']['size']=12
    fig.update_xaxes(tickfont=dict(size=12), title_font=dict(size=12))
    fig.for_each_yaxis(lambda axis: axis.update(tickfont=dict(size=12), title_font=dict(size=12)))
    fig.add_annotation(x=0.65, y=0.95, xref="paper", yref="paper", text=f"R-squared = {linear_regression_model.rsquared:.3f}", showarrow=False, font=dict(size=8))
    fig.add_annotation(x=0.65, y=0.90, xref="paper", yref="paper", text=f"f(x) = {slope:.3f} * x + {intercept:.3f}", showarrow=False, font=dict(size=8))
    fig.add_annotation(x=0.65, y=0.85, xref="paper", yref="paper", text=f"Correlation = {correlation:.3f}", showarrow=False, font=dict(size=8))
        
    lag = regression_models_df.loc[regression_models_df['Factor'] == factor, 'Optimal Lag'].values[0]

    if cumulative_performance:
        fig.update_layout(title_text=f"Linear Regression Analysis and Model of {time_basis} Returns vs. {factor} (lag={lag}))")
    else:
        
        fig.update_layout(title_text=f"Linear Regression Analysis and Model of {time_basis} Returns vs. {factor} {time_basis} Change (%) (lag={lag}))")
    
    fig.show()
    
    return fig

def check_stationarity(series):
    """
    Perform Dickey-Fuller test and check for stationarity of a given series.
    """
    result = adfuller(series)
    return result[1] <= 0.05  # If p-value is less than 0.05, the series is stationary

def create_var_model(data, maxlag=6, vif_threshold=5):
    # Ensure data is stationary
    non_stationary_columns = [column for column in data.columns if not check_stationarity(data[column])]
    if non_stationary_columns:
        # Difference non-stationary series
        data[non_stationary_columns] = data[non_stationary_columns].diff()
        # Drop the first row in all series
        data = data.dropna()

    threshold = 0.001 # if any columns with less than a 0.1% standard deviation, just drop it as more or less constant
    near_constant_columns = data.columns[data.std() < threshold]
    print('Near constant columns:', near_constant_columns)
    data = data.drop(columns=near_constant_columns)

    # Get the list of columns to drop based on high correlation to each other
    columns_to_drop = check_multicollinearity(data, vif_threshold)
    if columns_to_drop is None:
        columns_to_drop = []
    print(f'After check_multicollinearity(), columns to drop: {columns_to_drop}')

    # Add columns with high correlation to the columns to drop list
    corr_matrix = data.corr()
    for i in range(len(data.columns)):
        for j in range(i+1, len(data.columns)):
            if abs(corr_matrix.iloc[i,j]) > 0.8:
                col_i = data.columns[i]
                col_j = data.columns[j]
                if col_i not in columns_to_drop and col_j not in columns_to_drop:
                    print(f'Adding columns {col_i} and {col_j} to columns_to_drop')
                    columns_to_drop.append(col_i)
                    columns_to_drop.append(col_j)

    # Create the VAR model with all columns
    best_model = None
    best_aic = np.inf
    try:
        # Create the VAR model
        model = VAR(data)

        # Fit the model with the optimal lag order
        results = model.fit(ic='aic')
        print(f'VAR model results:\n{results.summary()}')

        # Update the best model if the AIC is lower
        if results.aic < best_aic:
            print(f'New best model found with AIC: {results.aic}')
            best_model = results
            best_aic = results.aic
    except Exception as e:
        print(f'Failed to create VAR model with all columns. Error: {e}')

    # if 'Portfolio' in columns_to_drop:, remove it
    if 'Portfolio' in columns_to_drop:
        columns_to_drop.remove('Portfolio')
        print(f'Removing Portfolio from columns_to_drop, leaving these: {columns_to_drop}')

    # Iterate through the columns to drop and create a VAR model with the optimal lag order
    for i in range(len(columns_to_drop) + 1):
        try:
            columns = columns_to_drop[:i]
            print(f'Creating VAR model with columns dropped: {columns}')
            # Drop the specified columns
            data_dropped = data.drop(columns=columns)

            print(f'After dropping columns, data.keys(): {data_dropped.keys()}\ndata.head():\n{data_dropped.head()}\ndata.tail():\n{data_dropped.tail()}')
            # Create the VAR model
            model = VAR(data_dropped)

            # Fit the model with the optimal lag order
            results = model.fit(ic='aic')
            print(f'VAR model results:\n{results.summary()}')

            # Update the best model if the AIC is lower
            if results.aic < best_aic:
                print(f'New best model found with AIC: {results.aic}')
                best_model = results
                best_aic = results.aic
        except Exception as e:
            print(f'Failed to create VAR model with columns dropped: {columns}. Error: {e}')

    if best_model is not None:
        print(f'Best model AIC: {best_aic}')
        print(f'Columns dropped: {columns_to_drop[:i]}')
        coefficients = best_model.params
        lag_order = best_model.k_ar
        return best_model, coefficients, lag_order
    else:
        print('Failed to create any VAR model.')
        return None, None, None
    


def plot_irf(var_model, periods=10):
    irf = var_model.irf(periods=periods)
    portfolio_index = var_model.names.index('Portfolio')
    data = []
    max_impact = 0
    max_impact_var = ''

    # Find the variable with the maximum cumulative absolute impact on the portfolio
    for i, name in enumerate(var_model.names):
        if i != portfolio_index:
            impact = irf.orth_irfs[:, i, portfolio_index]
            cumulative_impact = sum(abs(imp) for imp in impact)
            if cumulative_impact > max_impact:
                max_impact = cumulative_impact
                max_impact_var = name

    # Create a trace for each variable's impact on the portfolio
    for i, name in enumerate(var_model.names):
        if i != portfolio_index:
            impact = irf.orth_irfs[:, i, portfolio_index]
            color = colorsys.hsv_to_rgb(i / len(var_model.names), 0.5, 0.5)
            
            if name == max_impact_var:
                line_dict = {'color': 'rgb' + str(color), 'width': 3, 'dash': 'dash'}
                mode_dict = 'lines+markers'
            else:
                line_dict = {'color': 'rgb' + str(color), 'width': 1}
                mode_dict = 'lines'
                
            trace = go.Scatter(x=list(range(periods+1)), y=impact, mode=mode_dict, line=line_dict, name=f'Impact of {name} on Portfolio')
            data.append(trace)
    
    # Create the layout for the plot
    layout = go.Layout(title='Impulse Response of Portfolio Returns to Shocks in Macro Factors',
                       xaxis=dict(title='Period (portfolio return lag in months due to >1 std deviation change in factor)'),
                       yaxis=dict(title='Change in Portfolio Returns (%)'))

    # Create the figure and plot the traces
    fig = go.Figure(data=data, layout=layout)
    fig.update_yaxes(tickformat=".1%")
    fig.show()


def forecast_var_model(var_model, steps=5):
    # Forecast the next steps
    forecast = var_model.forecast(var_model.y, steps)

    # Find the index of the Portfolio variable
    portfolio_index = var_model.names.index('Portfolio')

    # Create the forecast series for the portfolio
    forecast_portfolio = forecast[:, portfolio_index]

    # Create a trace for the forecast
    trace = go.Scatter(x=list(range(len(forecast_portfolio))), y=forecast_portfolio, name='Forecast')

    # Create the layout for the plot
    layout = go.Layout(title='Forecast for Portfolio', xaxis=dict(title='Period'), yaxis=dict(title='Value'))

    # Create the figure and plot the trace
    fig = go.Figure(data=[trace], layout=layout)
    fig.show()


def calculate_optimal_lag(input_df, factor, maxlag=12):
    """Compute cross-correlation between portfolio returns and several lagged versions of the factor, and return the lag with the highest absolute cross-correlation"""
    df = input_df.copy()
    df = df.dropna(how='any').reset_index(drop=True)
    
    optimal_lag = 0
    max_cross_correlation = 0
    for lag in range(-maxlag, maxlag + 1):
        cross_correlation = abs(np.correlate(df['Portfolio'], df[factor].shift(lag), mode='valid').mean())
        if cross_correlation > max_cross_correlation:
            optimal_lag = lag
            max_cross_correlation = cross_correlation
    return optimal_lag

def create_regression_models_v0(returns_data, time_basis, cumulative_performance=False):
    monthly_input_data_df = prepare_data(returns_data)
    print(f'monthly_input_df index type: {type(monthly_input_data_df.index)}')
    
    factor_list = get_factor_list(monthly_input_data_df)
    print(f'monthly_input_df keys: {monthly_input_data_df.keys()}')
    print(f'monthly_input_df data head:\n{monthly_input_data_df.head()}')

    # Initialize dict to store models
    regression_models_dict = {}

    # Loop through all macro factors and build a regression model for each
    for factor in factor_list:
        regression_models_dict[factor] = {}
        # Create a dataframe for the current factor and the portfolio returns
        df_factor = monthly_input_data_df[['Portfolio', factor]].dropna()
        
        #print(f'creating linear regression model for *{factor}* data:\n{df_factor.head()}')

        # Calculate optimal lag for the current factor - a change in macro factor may take some time to affect the portfolio returns
        # The optimal lag is the lag with the highest absolute cross-correlation between the factor and the portfolio returns
        optimal_lag = calculate_optimal_lag(df_factor, factor)
        
        if optimal_lag > 0:
            print(f'Optimal lag for {factor} is {optimal_lag}')
            df_factor[factor] = df_factor[factor].shift(optimal_lag).dropna()
            
        # Calculate linear regression models for each factor
        regression_model, coefficient, p_value = calculate_linear_regression_model_for_factor(df_factor, factor)

        # Store the models, coefficients, and p-values in the dict
        regression_models_dict[factor]['linear'] = {"model": regression_model, "coefficient": coefficient, "p_value": p_value, "optimal_lag": optimal_lag}
       
        create_linear_regression_plots(monthly_input_data_df, factor, regression_models_dict, time_basis, cumulative_performance)
        
    plot_linear_regression_results(regression_models_dict)    

    # Use the multivariate model to identify significant factors
    multivariate_model, significant_features = calculate_multivariate_analysis(monthly_input_data_df)
    regression_models_dict['multivariate'] = {"model": multivariate_model, "significant_features": significant_features}   
    plot_multivariate_results(monthly_input_data_df, multivariate_model, significant_features)
    
    # Create and store the VAR model separately - only works with stationary data, i.e., the month to month change/returns data
    if not cumulative_performance:
        try:         
            print(f'\n~~~\nabout to create var model with monthly_input_df index type: {type(monthly_input_data_df.index)}\n~~~\n')       
            var_model, coefficients, lag_order = create_var_model(monthly_input_data_df)
            if var_model is not None:
                regression_models_dict['VAR'] = {"model": var_model, "coefficients": coefficients, "lag_order": lag_order}
                plot_irf(var_model, periods=10)  # Plot the impulse response function for 10 periods
                forecast_var_model(var_model, steps=5)
        except Exception as e:
            print(f'VAR model failed to run\n{e}')
    
    return regression_models_dict


def create_regression_models(returns_data, time_basis, cumulative_performance=False):
    monthly_input_data_df = prepare_data(returns_data)
    
    factor_list = get_factor_list(monthly_input_data_df)

    # Initialize dataframes to store models
    regression_models_df = pd.DataFrame(columns=['Model Type', 'Factor', 'Model', 'Coefficient', 'P-value', 'Optimal Lag'])
    multivariate_models_df = pd.DataFrame(columns=['Model Type', 'Model', 'Significant Features'])
    var_models_df = pd.DataFrame(columns=['Model Type', 'Model', 'Coefficients', 'Lag Order'])

    # Loop through all macro factors and build a regression model for each
    for factor in factor_list:
        # Create a dataframe for the current factor and the portfolio returns
        df_factor = monthly_input_data_df[['Portfolio', factor]].dropna()
        
        # Calculate optimal lag for the current factor - a change in macro factor may take some time to affect the portfolio returns
        # The optimal lag is the lag with the highest absolute cross-correlation between the factor and the portfolio returns
        optimal_lag = calculate_optimal_lag(df_factor, factor)
        
        if optimal_lag > 0:
            df_factor[factor] = df_factor[factor].shift(optimal_lag).dropna()
            
        # Calculate linear regression models for each factor
        regression_model, coefficient, p_value = calculate_linear_regression_model_for_factor(df_factor, factor)

        regression_models_df.loc[len(regression_models_df)] = {
            'Model Type': 'linear',
            'Factor': factor,
            'Model': regression_model, 
            'Coefficient': coefficient, 
            'P-value': p_value, 
            'Optimal Lag': optimal_lag}
        
#        create_linear_regression_plots(monthly_input_data_df, factor, regression_models_df, time_basis, cumulative_performance)
        
    # Use the multivariate model to identify significant factors
    multivariate_model, significant_features = calculate_multivariate_analysis(monthly_input_data_df)

    multivariate_models_df.loc[len(multivariate_models_df)] = {
        'Model Type': 'multivariate',
        'Model': multivariate_model, 
        'Significant Features': significant_features
    }

    # Create and store the VAR model separately - only works with stationary data
    if not cumulative_performance:
        try:         
            var_model, coefficients, lag_order = create_var_model(monthly_input_data_df)
            if var_model is not None:
                var_models_df.loc[len(var_models_df)] = {
                    'Model Type': 'VAR',
                    'Model': var_model, 
                    'Coefficients': coefficients, 
                    'Lag Order': lag_order}
#                plot_irf(var_model, periods=10)  # Plot the impulse response function for 10 periods
#                forecast_var_model(var_model, steps=5)
            print(f'\n~~~\nvar model created\n~~~\n')
            print(f'coefficients:\n{coefficients}\n\n{type(coefficients)}***********')
            print(f'lag order:\n{lag_order}\n\n***********')
            print(f'var model summary:\n{var_model.summary()}')
        except Exception as e:
            print(f'VAR model failed to run\n{e}')
    
    return regression_models_df, multivariate_models_df, var_models_df

tickers = ['AAPL','AMZN','NVDA','MMC','GOOG','MSFT','BTC-USD','ETH-USD','XOM','BAC','V','GOLD','^GSPC']
start_date = '2010-01-01'
end_date = datetime.now() - timedelta(1)
    
regression_models_df, multivariate_models_df, var_models_df = create_regression_models(cumulative_returns_data, 'Monthly', True)
regression_models_df, multivariate_models_df, var_models_df  = create_regression_models(returns_data, 'Monthly', False)

What we're trying to do, essentially, is use economic factors to understand and forecast a portfolio's returns. There are certainly examples of investment firms using similar methods.

Let's break this down into five steps:

1. Regression Analysis on These Macro Economic Factors vs a Weighted Portfolio of Stocks

This first step is crucial in understanding how each factor individually impacts the portfolio returns. Your calculate_linear_regression_models_from_macro_data_per_factor function seems to accomplish this. For each factor, it fits a linear regression model with the portfolio return as the dependent variable and the factor as the independent variable.

Just make sure your factors are at the same frequency and aligned by the dates.

2. Multivariate Analysis

After looking at each factor independently, you are right to consider the interaction between factors using a multivariate analysis.

Your function already includes this as well with the "all_factors" model. This model uses all factors as independent variables to predict the dependent variable.

You may also want to consider checking multicollinearity between the factors using Variance Inflation Factor (VIF) before including all the factors in your multivariate model. If there's high multicollinearity, it may lead to an unstable model where small changes in the data lead to large changes in the model. In that case, you may want to use a subset of the factors or consider dimensionality reduction techniques like PCA.

3. Select Factors for Prophet Forecast

With the output of the regression analyses, you can examine the coefficients and p-values of the factors to decide which ones are significant in predicting the portfolio returns.

You would typically choose factors with a low p-value (p<0.05) indicating statistical significance and a high coefficient absolute value indicating a stronger effect.

4. Forecast Macro Indicators & Predict Portfolio Value

Once you have selected the significant factors, you can forecast these using your tuned Prophet models.

After forecasting the factors, you can use these forecasts as input to the regression model to predict the future value of the portfolio.

5. Use Prophet to Predict the Portfolio

Finally, you would like to use Prophet to predict the value of the portfolio itself using the macro indicators chosen as regressors. This can be done by fitting a Prophet model on your portfolio data and adding the selected macro indicators as regressors using the add_regressor method.


Data Preparation: Before feeding data to a model, it needs to be prepared in a manner that's consistent with the model's expectations. This includes checking for missing values and possibly imputing them, scaling the data if necessary, and finally transforming the data into a format suitable for time series forecasting. In your case, this would mean having rows for each day (even if the macroeconomic data doesn't change on a daily basis), and columns for your independent variables (the macroeconomic indicators) and your dependent variable (the portfolio returns).

Feature Engineering: You could include lagged versions of your independent variables as new features. For instance, you could include the GDP from one month ago, two months ago, etc., as new columns in your dataset. This could help your model understand the relationship between past values of the independent variables and the current value of the dependent variable.

Model Selection: There are a number of models you can choose from for your forecasting. ARIMA models and variants thereof (like SARIMA, which accounts for seasonality) are traditional choices for this kind of task, but you could also use machine learning models like Random Forests, Support Vector Regression, or even deep learning models like LSTMs or 1D Convolutional Neural Networks. Ensemble methods, where you combine the predictions of multiple models, can also be very effective.

Model Training: Train your selected model on your training data.

Hyperparameter Tuning: Depending on the model you've chosen, there may be a number of hyperparameters that need to be set. You can use grid search or random search methods to find the combination of hyperparameters that yields the best performance on your validation set.

Evaluation: Evaluate the performance of your model on your test set. This gives you an unbiased estimate of how your model is likely to perform on unseen data.

Forecasting: Use your trained model to make forecasts for the next 365 days. Given that you want to make forecasts for multiple periods, you could either train a new model for each period (using all the data up to that period), or you could use a rolling forecast origin method, where you continually update your forecasts as new data becomes available.

Handling Seasonality: Models like SARIMA or Prophet (from Facebook) can handle seasonality internally. If you're using a model that doesn't, you may need to manually account for it in your data preparation step, possibly by detrending and deseasonalizing your data.

Model Updating: Given the nature of financial data, the relationships between variables may change over time. You should plan to periodically retrain your model with the most recent data to keep its performance optimal.

Uncertainty Estimation: Finally, it's important to have some measure of the uncertainty around your forecasts. Some models can provide this automatically (e.g., ARIMA, Prophet), but for others you might need to use methods like bootstrapping to generate prediction intervals.


## the data:
monthly_returns = {'Portfolio': portfolio_monthly_returns, 'S&P500': sp500_monthly_returns, 'Macro': mom_macro_data}
quarterly_returns = {'Portfolio': portfolio_quarterly_returns, 'S&P500': sp500_quarterly_returns, 'Macro': qoq_macro_data}
annual_returns = {'Portfolio': portfolio_annual_returns, 'S&P500': sp500_annual_returns, 'Macro': yoy_macro_data}

cumulative_monthy_returns = {'Portfolio': portfolio_cumulative_monthly_returns, 'S&P500': sp500_cumulative_monthly_returns, 'Macro': monthly_macro_data}
cumulative_quarterly_returns = {'Portfolio': portfolio_cumulative_quarterly_returns, 'S&P500': sp500_cumulative_quarterly_returns, 'Macro': quarterly_macro_data} 
cumulative_annual_returns = {'Portfolio': portfolio_cumulative_annual_returns, 'S&P500': sp500_cumulative_annual_returns, 'Macro': yoy_macro_data}        

## tuned hyperparameter file
import glob
from datetime import datetime
import pandas as pd
import os

date = datetime.now().strftime("%Y%m%d")
print(f'date: {date}')

cur_tuned_params_df = pd.read_csv(f'tuned_macro_hyperparameters_20230605.csv', index_col=[0, 1])

print(f'cur_tuned_params_df:\n{cur_tuned_params_df}')

def load_latest_tuned_hyperparameters():
    # List all files that begin with "tuned_macro_hyperparameters"
    files = glob.glob('tuned_macro_hyperparameters_*.csv')

    # If no files found, return None
    if not files:
        return None

    # Sort files by date
    files.sort(key=os.path.getmtime, reverse=True)

    # Load the most recent file
    latest_file = files[0]
    df = pd.read_csv(latest_file, index_col=[0, 1])

    return df

cur_tuned_params_df = load_latest_tuned_hyperparameters()
print(f'cur_tuned_params_df:\n{cur_tuned_params_df}')

# TODO: flag the tuned parms as good or needing more work

In [ ]:
import os, sys
# enable absolute paths transversal (from notebooks folder to src folder)
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
import glob
from datetime import datetime
import pandas as pd
import os
import json

from datetime import datetime, timedelta

from pypfopt import expected_returns, EfficientFrontier
import pandas as pd
import numpy as np

import src.utils as utils
import src.macro.calculate as calculate
import src.macro.plot as plot

tickers = ['AAPL','AMZN','NVDA','MMC','GOOG','MSFT','BTC-USD','ETH-USD','XOM','BAC','V','GOLD','^GSPC']
start_date = '2014-01-01'
end_date = datetime.now() - timedelta(1)

#cur_tuned_params_dict = load_latest_tuned_hyperparameters()
#tuned_macro_hyperparameters_tuned_hyper_parms_w_init_2190_period_68.4375_horizon_136.875_20230608.json
def load_tuned_hyperparameters(file_name):

    df = pd.read_json(file_name)

    return df

cur_tuned_params_dict = load_tuned_hyperparameters('tuned_macro_hyperparameters_tuned_hyper_parms_w_init_2190_period_68.4375_horizon_136.875_20230608.json')

# monthly_returns = {'Portfolio': portfolio_monthly_returns, 'S&P500': sp500_monthly_returns, 'Macro': mom_change_in_macro_data} # compare macro with *cumulative* monthly returns

returns_data, cumulative_returns_data = get_combined_returns_data(tickers, start_date, end_date)
print(f"monthly portfolio returns {returns_data['Monthly']['Portfolio'].head()}")
print(f"monthly sp500 returns {returns_data['Monthly']['S&P500'].head()}")
print(f"monthly macro keys {returns_data['Monthly']['Macro'].keys()}")
print(f"monthly macro CPI {returns_data['Monthly']['Macro']['CPI'].head()}")

In [ ]:
from functools import reduce
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.api import OLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

import warnings
import plotly.graph_objects as go
import plotly.subplots as sp
import itertools


warnings.filterwarnings('ignore')

def create_altair_plots(df, factor, regression_model, factor_forecast_model, months_to_forecast):
    monthly_portfolio_forecast, cumulative_monthly_portfolio_forecast = forecast_portfolio_from_regression_based_on_predicted_factor(factor, regression_model, factor_forecast_model, months_to_forecast)
    
    # Define the base plot
    base = alt.Chart(df.reset_index()).encode(x='index:T')
    
    # Create the Returns plot
    returns_layer = base.mark_line().encode(
        y='Portfolio',
        color=alt.value('royalblue')
    ) + base.mark_line().encode(
        y='S&P500',
        color=alt.value('grey')
    ) + base.mark_line().encode(
        y=factor,
        color=alt.value('black')
    )
    
    # Create the Regression plot
    X = add_constant(df[[factor]])
    fitted_values = regression_model.predict(X)
    regression_layer = base.mark_point().encode(
        y='Portfolio',
        x=factor
    ) + base.mark_line().encode(
        y=alt.Y('fitted_values', title='Fitted Line'),
        x=factor
    )
    
    # Create the Residuals plot
    residuals = df["Portfolio"] - fitted_values
    residuals_layer = base.mark_line().encode(
        y=alt.Y('residuals', title='Linear Regression Residuals')
    )

    # Create the Forecast plot
    forecast_layer = alt.Chart(monthly_portfolio_forecast.reset_index()).mark_line().encode(
        x='index:T',
        y=factor
    ) + alt.Chart(monthly_portfolio_forecast.reset_index()).mark_line().encode(
        x='index:T',
        y='Portfolio'
    )

    # Create the Cumulative Returns Forecast plot
    cumulative_layer = alt.Chart(cumulative_monthly_portfolio_forecast.reset_index()).mark_line().encode(
        x='index:T',
        y=factor
    ) + alt.Chart(cumulative_monthly_portfolio_forecast.reset_index()).mark_line().encode(
        x='index:T',
        y='Portfolio'
    )

    # Concatenate the plots and customize
    chart = alt.hconcat(returns_layer, regression_layer, residuals_layer, forecast_layer, cumulative_layer).properties(
        title=f'Return, Regression, Residuals, Forecast, and Cumulative Forecast for {factor}'
    )
    
    # Display the plot
    chart.display()
    
def plot_multivariate_results(input_df, multivariate_model, significant_features):
    fig = go.Figure(data=[go.Bar(name='Coefficient', x=multivariate_model.params.index, y=multivariate_model.params.values), 
                          go.Bar(name='p-value', x=multivariate_model.pvalues.index, y=multivariate_model.pvalues.values)])
    fig.update_layout(barmode='group', title_text='Multivariate Regression Results')
    fig.show()

    # Scatter plots for each significant feature
    for feature in significant_features:
        # Calculate the OLS trendline
        m, b = np.polyfit(input_df[feature], input_df['Portfolio'], 1)

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=input_df[feature], y=input_df['Portfolio'], mode='markers', name='observations'))
        fig2.add_trace(go.Scatter(x=input_df[feature], y=m*input_df[feature] + b, mode='lines', name='OLS trendline'))
        fig2.update_layout(title=f'Portfolio Returns vs {feature}', xaxis_title=feature, yaxis_title='Portfolio Returns')
        fig2.show()

def plot_linear_regression_results(regression_model_dict):
    fig = go.Figure()
    
    for factor, result in regression_model_dict.items():
        fig.add_trace(
            go.Bar(name=f'Coefficient - {factor}', x=[factor], y=[result['coefficient']], marker_color='lightblue')
        )
        fig.add_trace(
            go.Bar(name=f'p-value - {factor}', x=[factor], y=[result['p_value']], marker_color='lightsalmon')
        )

    fig.update_layout(barmode='group', title_text='Linear Regression Results')
    fig.show()
    
# Create a function to visualize backtesting results
def plot_backtesting_results(mse, mae, forecast, holdout):
    fig = go.Figure()

    # Plot the actual data
    fig.add_trace(go.Scatter(x=holdout.index, y=holdout['Portfolio'], mode='lines', name='Actual Portfolio'))

    # Plot the forecasted data
    fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, mode='lines', name='Predicted Portfolio', line=dict(color='royalblue')))

    # Plot the confidence interval
    fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat_lower, fill='tonexty', mode='lines', fillcolor='rgba(68,68,68,0.3)', line_color='transparent', showlegend=False))
    fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat_upper, fill='tonexty', mode='lines', fillcolor='rgba(68,68,68,0.3)', line_color='transparent', showlegend=False))

    # Add MSE and MAE as annotations
    fig.add_annotation(
        x=0.05,
        y=0.95,
        xref="paper",
        yref="paper",
        text=f"MSE: {mse:.3f}, MAE: {mae:.3f}",
        showarrow=False,
        font=dict(
            size=10
        ),
        bgcolor="rgba(255,255,255,0.8)"
    )

    fig.update_layout(title='Backtesting Results', xaxis_title='Date', yaxis_title='Portfolio Value')
    fig.show()


    
def plot_prophet_forecast(forecast):
    fig = go.Figure()

    # Historical data
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['y'], mode='lines', name='Historical Returns', line=dict(color='royalblue')))

    # Forecasted data
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecasted Returns', line=dict(color='darkorange')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], mode='lines', name='Lower Bound', line=dict(color='grey'), fill='tonexty'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], mode='lines', name='Upper Bound', line=dict(color='grey'), fill='tonexty'))

    fig.update_layout(
        title='Prophet Model Forecast of Portfolio Returns',
        xaxis_title='Date',
        yaxis_title='Returns',
        hovermode='x',
        template='plotly_dark'
    )

    fig.show()
    
    
def create_comparison_regress_forecast_plots(df, factor, regression_model, factor_forecast_model, months_to_forecast):

    fig = sp.make_subplots(rows=1, cols=5, subplot_titles=("Returns", "Regression", "Residuals", "Forecast"), specs=[[{'secondary_y': True}, {}, {}, {}, {}]])

    # plot the original data to compare portfolio vs benchmark vs macro economic factor
    fig.add_trace(go.Scatter(x=df.index, y=df["Portfolio"], mode='lines', name='Portfolio', line=dict(color='royalblue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df["S&P500"], mode='lines', name='S&P500', line=dict(color='grey')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df[factor], mode='lines', name=factor, line=dict(dash='dot')), secondary_y=True, row=1, col=1)
    fig.update_yaxes(title_text="Returns", row=1, col=1)
    fig.update_yaxes(title_text=factor, secondary_y=True, row=1, col=1)
    
    # Plot the linear regression results of the monthly portfolio returns vs the monthly change in macroeconomic factor
    X = add_constant(df[[factor]])
    fitted_values = regression_model.predict(X)
    fig.add_trace(go.Scatter(x=df[factor], y=df["Portfolio"], mode='markers', name=f'{factor} (x) vs Returns (y)'), row=1, col=2)
    fig.add_trace(go.Scatter(x=df[factor], y=fitted_values, mode='lines', name='Fitted line'), row=1, col=2)
    fig.update_xaxes(title_text=f'{factor} Monthly Change', tickformat=".2%", row=1, col=2)
    fig.update_yaxes(title_text="Returns", tickformat=".2%", row=1, col=2)
    
    # Plot the linear regression residuals
    residuals = df["Portfolio"] - fitted_values
    fig.add_trace(go.Scatter(x=df.index, y=residuals, mode='lines', name='Linear Regression Residuals'), row=1, col=3)
    
    # forecast the portfolio returns based on the linear regression prediction model leveraging the prophet forecast of the future monthly change in the macroeconomic factor
    monthly_portfolio_forecast, cumulative_monthly_portfolio_forecast = forecast_portfolio_from_regression_based_on_predicted_factor(factor, regression_model, factor_forecast_model, months_to_forecast)
        
    # create a plot of the linear regression prediction of monthly returns based on Prophet's forecast of the future monthly change in the macroeconomic factor
    fig.add_trace(go.Scatter(x=monthly_portfolio_forecast.index, y=monthly_portfolio_forecast[f'{factor}'], mode='lines', name=f'{factor} (Time Series Prediction)', line=dict(color='red')), row=1, col=4)
    fig.add_trace(go.Scatter(x=monthly_portfolio_forecast.index, y=monthly_portfolio_forecast, mode='lines', name='Portfolio Monthly Returns (Linear Regression Prediction)', line=dict(color='red')), row=1, col=4)

    # create a plot of the cumulative returns based on Prophet's forecast of the future monthly change in the macroeconomic factor
    fig.add_trace(go.Scatter(x=cumulative_monthly_portfolio_forecast.index, y=cumulative_monthly_portfolio_forecast[f'{factor}'], mode='lines', name=f'{factor} Cumulative Time Series Forecast',  line=dict(dash='dot')), row=1, col=5)
    fig.add_trace(go.Scatter(x=cumulative_monthly_portfolio_forecast.index, y=cumulative_monthly_portfolio_forecast['Portfolio'], mode='lines', name='Cumulative Portfolio Returns Forecast (Linear Regression Prediction)', line=dict(color='royalblue')), row=1, col=5)

    fig.update_layout(title_text=f'Portfolio Returns vs {factor}, Linear Regression of Portfolio vs {factor}, and Time Series Forecast for {factor} with Linear Regression Prediction of Portfolio Returns', showlegend=False)
    fig.show()
    

    
def validate_data(data, trace=None):
    if type(data) == pd.DataFrame:
        nan_columns = data.columns[data.isna().any()].tolist()
        inf_columns = data.columns[np.isinf(data).any()].tolist()

        print(f"{trace}: Columns with NaN values: {nan_columns}")
        print(f"{trace}: Columns with Inf values: {inf_columns}")
    else:
        # must be a pd.Series, validate that no values are NaN or Inf
        if data.isna().any():
            print(f"{trace}: Series has NaN values")
        if np.isinf(data).any():
            print(f"{trace}: Series has Inf values")
    
# Aligning the dataframes
def align_dataframes(dfs):
    # Get the common datetime index
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)
    
    # Align all dataframes to the common index
    aligned_dfs = [df.loc[common_index] for df in dfs]
    return aligned_dfs

def prepare_data(returns_data):
    portfolio_returns = returns_data['Monthly']['Portfolio'].to_frame(name='Portfolio')
    sp500_returns = returns_data['Monthly']['S&P500'].to_frame(name='S&P500')

    # concat sp500 with portfolio returns
    portfolio_returns = pd.concat([portfolio_returns, sp500_returns], axis=1)

    # Loop through all macro factors and add to DataFrame
    for factor_name, factor_df in returns_data['Monthly']['Macro'].items():
        factor_df = factor_df.rename(factor_name)  # Rename the series
        portfolio_returns = pd.concat([portfolio_returns, factor_df], axis=1)

    # Use align_dataframes to ensure the data is properly aligned
    aligned_data = align_dataframes([portfolio_returns])

    aligned_data[0] = aligned_data[0].dropna()

    return aligned_data[0]  # align_dataframes returns a list, so we take the first element

"""
The check_multicollinearity() function calculates the Variance Inflation Factor (VIF) for each macro factor. 
VIF measures the correlation between each factor and all other factors. If a factor's VIF exceeds a certain 
threshold (in this case, 5), it indicates high multicollinearity.
"""
def check_multicollinearity(df, vif_threshold=5):
    variables = df.columns
    vif_df = pd.DataFrame()
    vif_df["VIF"] = [variance_inflation_factor(df[variables].values, df.columns.get_loc(var)) for var in df.columns]
    vif_df["Features"] = variables
    while vif_df[vif_df['VIF'] > vif_threshold].any(axis=None):
        remove = vif_df.sort_values('VIF',ascending=0)['Features'][:1]
        df.drop(remove,axis=1,inplace=True)
        variables = df.columns
        vif_df = pd.DataFrame()
        vif_df["VIF"] = [variance_inflation_factor(df[variables].values, df.columns.get_loc(var)) for var in df.columns]
        vif_df["Features"] = variables
    return df

def get_factor_list(df):
    factor_list = list(df.columns)
    factor_list.remove('Portfolio')
    factor_list.remove('S&P500')
    return factor_list

def calculate_linear_regression_model_for_factor(df, factor):
    X = df[[factor]]
    y = df['Portfolio']
    X2 = add_constant(X)
    model = OLS(y, X2).fit()
    coefficient = model.params[factor]
    p_value = model.pvalues[factor]
    return model, coefficient, p_value

def calculate_multivariate_analysis(df):
    X = df[get_factor_list(df)]  
    y = df['Portfolio']
    
    model = OLS(y, add_constant(X)).fit()
    
    # Extract the significant features based on p-values
    significant_features = model.pvalues[model.pvalues < 0.05].index.tolist()
    
    if 'const' in significant_features:
        significant_features.remove('const')
        
    return model, significant_features

def tune_hyperparameters_for_portfolio_forecast(df, horizon):
   
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 5.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative'],
    }
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here
    smapes = []  # Store the sMAPEs for each params here
    coverages = []  # Store the Coverages for each params here
    
    # Use cross validation to evaluate all parameters
    for params in all_params:
        try:
            m = Prophet(**params).fit(df)  # Fit model with given params
            df_cv = cross_validation(m, horizon=f'{horizon} days', parallel="processes")
            df_p = performance_metrics(df_cv, rolling_window=1)
            rmses.append(df_p['rmse'].values[0])
            smapes.append(df_p['smape'].values[0])
            coverages.append(df_p['coverage'].values[0])
        except Exception as e:
            # print the exception
            print(f"Exception {e} in tuning hyper parms parameter: ", params)
 
    # Find the best parameters - use smape for now
    best_params_rmse = all_params[np.argmin(rmses)]
    best_params_smape = all_params[np.argmin(smapes)]
    best_params_coverage = all_params[np.argmax(coverages)]

    return best_params_smape

# Prophet Model
def build_prophet_model_for_portfolio(input_data, significant_factors, factor_forecast_models_dict, months_to_forecast):

    df = input_data['Portfolio']
    df = df.reset_index()
    df.columns = ['ds', 'y']
    #print(f'prophet portfolio model input data shape: {df.head()}')   
     
    # tune hyper parameters and do cross validations to select the optimal parameters based on smape
    hyperparameters = tune_hyperparameters_for_portfolio_forecast(df, int(months_to_forecast/10))
    
    model = Prophet(**hyperparameters)
    
    # use the forecast_models_dict to select the significant factors as regressors
    for factor in significant_factors:
        regressor = factor_forecast_models_dict[factor]['forecast'][['ds', 'yhat']]
        # ensure the data is in the correct format and dates aligned
        regressor.columns = ['ds', factor]
        model.add_regressor(factor)
    
    # build and fit the model to the
    model.fit(df)
    
    # define the period for which we want a prediction - most data is monthly, use MS for month start
    future = model.make_future_dataframe(periods=months_to_forecast, freq='MS')
    
    # merge the future data from each regressor's Prophet forecast with the main 'future' DataFrame
    for factor in significant_factors:
        future = future.merge(forecast_models_dict[factor]['forecast'][['ds', 'yhat']], on='ds', how='left')
        future = future.rename(columns={'yhat': factor})
    
    # use the model to make a forecast
    forecast = model.predict(future)
    
    return model, forecast

# Define the forecast function
def build_prophet_forecast_for_factor(df, periods, data_frequency, hyperparameters):
    
    #print(f'prophet_forecast df:\n{df.head()}')
    #print(f'prophet_forecast df:\n{df.keys()}')
    df = df.reset_index()
    df.columns = ['ds', 'y']
    df.dropna(inplace=True)
    #print(f'prophet_forecast df:\n{df.head()}')
    #print(f'prophet_forecast df:\n{df.keys()}')
            
    # Check for missing values
    if df['y'].isna().any():
        print("WARNING: The 'y' column contains missing values. These will be dropped.")
        df = df.dropna()

    # Check for non-numeric values
    if df['y'].apply(lambda x: not isinstance(x, (int, float))).any():
        print("WARNING: The 'y' column contains non-numeric values. These cannot be used in the model.")
    
    # Check for zero values
    if (df['y'] == 0).any():
        print("WARNING: The 'y' column contains zero values. These can cause problems for the model.")
    
    # Check for extreme values
    if df['y'].max() > 1e6 or df['y'].min() < -1e6:
        print("WARNING: The 'y' column contains very large or very small values. These can cause problems for the model.")

    # define the model
    model = Prophet(changepoint_prior_scale=hyperparameters['changepoint_prior_scale'], 
                    seasonality_prior_scale=hyperparameters['seasonality_prior_scale'], 
                    seasonality_mode=hyperparameters['seasonality_mode'])
       
    # fit the model
    model.fit(df)
    
    # define the period for which we want a prediction - most data is monthly, use MS for month start
    future = model.make_future_dataframe(periods=periods, freq=data_frequency)
    
    # use the model to make a forecast
    forecast = model.predict(future)
    
    return model, forecast

# Model Evaluation
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return mse, mae

# Backtest Model
def backtest_model(df, train_size, model):
    # Split the data
    train = df[:train_size]
    holdout = df[train_size:]

    # Make predictions
    future = model.make_future_dataframe(periods=len(holdout), freq='MS')
    forecast = model.predict(future)  # Adjusted line

    # Evaluate the model
    mse, mae = evaluate_model(holdout['Portfolio'], forecast['yhat'])
    return mse, mae, forecast, holdout  # Return forecast and holdout for visualization

def forecast_portfolio_from_regression_based_on_predicted_factor(factor, regression_model, forecast_model, months_to_forecast):

    # Predict the factor
    future_factor = forecast_model.predict(forecast_model.make_future_dataframe(periods=months_to_forecast, freq='MS'))

    # Add the predicted factor values into the portfolio DataFrame
    y = future_factor['yhat'].dropna()
    dates = future_factor['ds']  # Assuming 'ds' is a column with dates

    # Predict the portfolio based on the predicted factor
    X = add_constant(y)
    portfolio_pred = regression_model.predict(X)
    
    monthly_forecast_df = pd.DataFrame({'Portfolio': portfolio_pred, f'{factor}': y})
    monthly_forecast_df.set_index(dates, inplace=True)
                
    cumulative_monthly_returns = (1 + portfolio_pred).cumprod()
    cumulative_monthly_returns = cumulative_monthly_returns.rename('Portfolio')
    
    cumulative_factor_value = (1 + y).cumprod()
    cumulative_factor_value = cumulative_factor_value.rename(factor)

    # Build the forecast DataFrame - what is the dateindex set to?
    cumulative_forecast_df = pd.DataFrame({'Portfolio': cumulative_monthly_returns, f'{factor}': cumulative_factor_value})
    cumulative_forecast_df.set_index(dates, inplace=True)

    return monthly_forecast_df, cumulative_forecast_df

def create_linear_regression_plots_v0(df, factor, linear_regression_model, factor_forecast_model, months_to_forecast):
    X = df[[factor]]
    y = df['Portfolio']
    X2 = add_constant(X)
    predictions = linear_regression_model.predict(X2)
    residuals = y - predictions

    # Intercept and slope for the annotation
    intercept = linear_regression_model.params['const']
    slope = linear_regression_model.params[factor]
    
    #monthly_forecast_df, cumulative_forecast_df = forecast_portfolio_from_regression_based_on_predicted_factor(factor, linear_regression_model, factor_forecast_model, months_to_forecast)

    # Initialize subplots
    fig = sp.make_subplots(rows=1, cols=4, subplot_titles=("Portfolio vs SP500 vs Factor", "Predicted vs Actual Values", "Residuals vs Predicted Values"),  specs=[[{'secondary_y': True}, {'secondary_y': True}, {}, {'secondary_y': True}]])
    
    # Subplot 1: Portfolio vs SP500 vs Factor
    fig.add_trace(go.Scatter(x=df.index, y=df["Portfolio"], mode='lines', name='Portfolio', line=dict(color='royalblue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df["S&P500"], mode='lines', name='S&P500', line=dict(color='grey')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df[factor], mode='lines', name=factor, line=dict(dash='dot')), secondary_y=True, row=1, col=1)
    
    fig.update_yaxes(title_text="Returns", row=1, col=1)
    fig.update_yaxes(title_text=factor, secondary_y=True, row=1, col=1)
    
    # Subplot 2: Predicted vs Actual values 
    fig.add_trace(
        go.Scatter(
            x = y,
            y = predictions,
            mode = 'markers',
            marker=dict(
                size=10,
                color='rgba(152, 0, 0, .8)'
            ),
            name='Predicted vs Actual'
        ), row=1, col=2
    )
    # Predicted vs. Actual Portfolio Returns with Future Forecast
    #fig.add_trace(go.Scatter(x=monthly_forecast_df.index, y=monthly_forecast_df["Portfolio"], mode='lines', name='Portfolio Forecast', line=dict(color='orange')), row=1, col=2)
    # Factor Forecasts
    #fig.add_trace(go.Scatter(x=monthly_forecast_df.index, y=monthly_forecast_df[factor], mode='lines', name=f'{factor} Forecast', line=dict(color='green')), secondary_y=True, row=1, col=2)
    
    # Fitted line
    fig.add_trace(
        go.Scatter(
            x = y,
            y = intercept + slope * y,
            mode = 'lines',
            line = dict(color='red', width=2),
            name='Fitted line'
        ), row=1, col=2
    )
    
    # Subplot 3: Residuals vs Predicted Values
    fig.add_trace(
        go.Scatter(
            x = predictions,
            y = residuals,
            mode = 'markers',
            marker=dict(
                size=10,
                color='rgba(152, 0, 0, .8)'
            ),
            name='Residuals'
        ), row=1, col=3
    )

    # Adding a horizontal line at y=0
    fig.add_shape(
        type='line',
        line=dict(
            color='grey',
            width=2,
            dash='dash'),
        x0=min(predictions),
        x1=max(predictions),
        y0=0,
        y1=0,
        row=1,
        col=3
    )
    
    # Subplot 4: Cumulative Returns with subplot title
    #fig.add_trace(go.Scatter(x=cumulative_forecast_df.index, y=cumulative_forecast_df['Portfolio'], mode='lines', name='Cumulative Portfolio Returns', line=dict(color='purple')), row=1, col=4)
    #fig.add_trace(go.Scatter(x=cumulative_forecast_df.index, y=cumulative_forecast_df[factor], mode='lines', name=f'Cumulative {factor}', line=dict(color='cyan')), secondary_y=True, row=1, col=4)
    fig.update_yaxes(title_text="Cumulative Returns", row=1, col=4)
    fig.update_yaxes(title_text=f"Cumulative {factor}", secondary_y=True, row=1, col=4)

    # Annotations for R-squared and the function's equation
    fig.add_annotation(
        x=0.05,
        y=0.95,
        xref="paper",
        yref="paper",
        text=f"R-squared = {linear_regression_model.rsquared:.3f}",
        showarrow=False,
        font=dict(
            size=8
        ),
        xshift=-25  # adjust this value to move annotation left or right
    )

    fig.add_annotation(
        x=0.05,
        y=0.90,
        xref="paper",
        yref="paper",
        text=f"f(x) = {slope:.3f} * x + {intercept:.3f}",
        showarrow=False,
        font=dict(
            size=8
        ),
        xshift=-25
    )

    # Update layout
    fig.update_layout(title_text="Linear Regression Analysis", title_x=0.5)
    fig.show()

def subplot_1(fig, df, monthly_forecast_df, factor):

    # historical data
    fig.add_trace(go.Scatter(x=df.index, y=df["Portfolio"], mode='lines', name='Portfolio', line=dict(color='royalblue')), secondary_y=False)
    fig.add_trace(go.Scatter(x=df.index, y=df["S&P500"], mode='lines', name='S&P500', line=dict(color='grey')), secondary_y=False)
    fig.add_trace(go.Scatter(x=df.index, y=df[factor], mode='lines', name=factor, line=dict(dash='dot', color='red')), secondary_y=True)
    
    #fig.add_trace(go.Scatter(x=monthly_forecast_df.index, y=monthly_forecast_df["Portfolio"], mode='lines', name='Portfolio Forecast', line=dict(color='orange')), secondary_y=False)
    #fig.add_trace(go.Scatter(x=monthly_forecast_df.index, y=monthly_forecast_df[factor], mode='lines', name=f'{factor} Forecast', line=dict(color='green')), secondary_y=True)
    #fig.add_shape(type="line", x0=df.index[-1], x1=monthly_forecast_df.index[0], y0=0, y1=1, yref="paper", line=dict(color="Black", dash="dashdot"))
    fig.update_yaxes(title_text="Returns")
    fig.update_yaxes(title_text=factor, secondary_y=True)
    return fig

def subplot_2(fig, cumulative_forecast_df, factor):

    fig.add_trace(go.Scatter(x=cumulative_forecast_df.index, y=cumulative_forecast_df['Portfolio'], mode='lines', name='Cumulative Portfolio Returns', line=dict(color='purple')), row=1, col=3, secondary_y=False)
    fig.add_trace(go.Scatter(x=cumulative_forecast_df.index, y=cumulative_forecast_df[factor], mode='lines', name=f'Cumulative {factor}', line=dict(color='cyan')), row=1, col=3, secondary_y=True)
    fig.update_yaxes(title_text="Cumulative Returns")
    fig.update_yaxes(title_text=f"Cumulative {factor}", secondary_y=True)
    return fig

def subplot_3(fig, y, predictions):

    fig.add_trace(
        go.Scatter(
            x = y,
            y = predictions,
            mode = 'markers',
            marker=dict(
                size=10,
                color='rgba(152, 0, 0, .8)'
            ),
            name='Predicted vs Actual'
        ), row=1, col=5, secondary_y=False
    )
    return fig

def subplot_4(fig, predictions, residuals):

    fig.add_trace(
        go.Scatter(
            x = predictions,
            y = residuals,
            mode = 'markers',
            marker=dict(
                size=10,
                color='rgba(152, 0, 0, .8)'
            ),
            name='Residuals'
        ), row=2, col=5, secondary_y=False
    )
    fig.add_shape(
        type='line',
        line=dict(
            color='grey',
            width=2,
            dash='dash'),
        x0=min(predictions),
        x1=max(predictions),
        y0=0,
        y1=0
    )
    return fig

def create_linear_regression_plots(df, factor, linear_regression_model, factor_forecast_model, months_to_forecast):
    # Create fig object
    fig = sp.make_subplots(rows=2, cols=5, specs=[[{'rowspan': 2, 'colspan': 2, 'secondary_y': True}, None, {'rowspan': 2, 'colspan': 2, 'secondary_y': True}, None, {}], [None, None, None, None, {}]])

    X = df[[factor]]
    y = df['Portfolio']
    X2 = add_constant(X)
    predictions = linear_regression_model.predict(X2)
    residuals = y - predictions

    intercept = linear_regression_model.params['const']
    slope = linear_regression_model.params[factor]
    
    monthly_forecast_df, cumulative_forecast_df = forecast_portfolio_from_regression_based_on_predicted_factor(factor, linear_regression_model, factor_forecast_model, months_to_forecast)

    # Pass fig into each subplot function
    fig = subplot_1(fig, df, monthly_forecast_df, factor)
    fig = subplot_2(fig, cumulative_forecast_df, factor)
    fig = subplot_3(fig, y, predictions)
    fig = subplot_4(fig, predictions, residuals)
    
    fig.update_layout(title_text="Linear Regression Analysis", title_x=0.5)
    fig.show()

def run_model(returns_data, cumulative_returns_data, hyper_params_dict, months_to_forecast):
    input_data_df = prepare_data(returns_data)
    factor_list = get_factor_list(input_data_df)
    print(f'reg_data input_data_df: {input_data_df.keys()}')
    print(f'Regression data:\n{input_data_df.head()}')

    # Initialize dict to store models
    regression_models_dict = {}
    forecast_models_dict = {}

    # Loop through all macro factors and build a regression model for each
    for factor in factor_list:
        # Create a dataframe for the current factor and the portfolio returns
        df_factor = input_data_df[['Portfolio', factor]].dropna()
        #print(f'creating linear regression model for *{factor}* data:\n{df_factor.head()}')

        # Calculate linear regression models for each factor
        regression_model, coefficient, p_value = calculate_linear_regression_model_for_factor(df_factor, factor)

        # Store the models, coefficients, and p-values in the dict
        regression_models_dict[factor] = {"model": regression_model, "coefficient": coefficient, "p_value": p_value}

        factor_forecast_model, factor_forecast = build_prophet_forecast_for_factor(df_factor.drop(['Portfolio'], axis=1), periods=months_to_forecast, data_frequency='MS', hyperparameters=hyper_params_dict[factor]['Monthly Change']['smape'])

        # Store the forecast models and forecasts in the dict
        forecast_models_dict[factor] = {"model": factor_forecast_model, "forecast": factor_forecast}
        
        create_linear_regression_plots_v0(input_data_df, factor, regression_model, factor_forecast_model, months_to_forecast)

    plot_linear_regression_results(regression_models_dict)
    # Use the multivariate model to identify significant factors
    multivariate_model, significant_features = calculate_multivariate_analysis(input_data_df)
    plot_multivariate_results(input_data_df, multivariate_model, significant_features)
    
    # Build the final Prophet model using the significant factors as regressors
    #final_model, final_forecast = build_prophet_model_for_portfolio(input_data_df, significant_features, forecast_models_dict, months_to_forecast)
    #plot_prophet_forecast(final_forecast)
    
    # Backtest the model
    #mse, mae, forecast, holdout = backtest_model(input_data_df.reset_index().rename(columns={'index': 'ds', 'Portfolio': 'y'}), int(len(input_data_df)*0.8), final_model)
    #plot_backtesting_results(mse, mae, forecast, holdout)

    return regression_models_dict, forecast_models_dict #, final_model, final_forecast


regression_models_dict, forecast_models_dict = run_model(returns_data, cumulative_returns_data, cur_tuned_params_dict, months_to_forecast=12*10)


In [ ]:
def display_regression_formula(model, factor_name, y):
    # Get the intercept and coefficient
    intercept = model.intercept_
    coef = model.coef_[0]

    # Format the formula string
    formula = f"{y} = {intercept:.4f} + ({coef:.4f} * {factor_name})"

    # Display the formula
    print(formula)


